# MENTORÍA: Predicción de indicadores sociales para países del MERCOSUR

## Práctico 2: Análisis y Curación

[Link a la descripción del proyecto](https://github.com/jfrau/DiploDatos2020/blob/master/README.md)

### Objetivos

1. Lograr un entendimiento y análisis más profundo de los datos haciendo uso de herramientas estadísticas y partiendo de lo realizado en el práctico de Análisis y Visualización.
2. Realizar una limpieza y depuración del dataset (outliers, valores nulos, etc.) generando un nuevo dataset que contenga la información más fiable posible.
3. Focalizar el estudio de la predicción en Argentina.


# Introducción

En concordancia con los objetivos anteriores, en **líneas generales** se buscará:

- Seguir indagando y profundizando el conocimiento de los datos, 
- Descubrir más patrones,
- Confirmar teorías que surgieron durante el primer análisis y 
- Eliminar información ruidosa y no relevante. 

En **líneas más específicas**, centraremos el análisis en Argentina aprovechando nuestro conocimiento de dominio pero recurriremos a los datos de los demás países cada vez que sea necesario.

El **resultado final del práctico** deberá ser un nuevo dataset por Indicador que solo incluya datos de Argentina, lo más limpio y fiable posible (por ejemplo, sin valores nulos).

En base a lo anterior se proponen las siguientes consignas. **Recuerden que las mismas son disparadoras y que cualquier enfoque adicional desde su formación, conocimiento de dominio o simple curiosidad siempre enriquecerá el trabajo.**

# Dataset Originales

In [ ]:
#Importacion de las librerias

import pandas as pd
#Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap



In [ ]:
# Para que pandas no trunque aquellas celdas con contenido muy largo (como textos)
pd.set_option('display.max_colwidth', -1)

# Para que pandas no trunque la vista de todas las columnas
pd.set_option('display.max_columns', None)

In [ ]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
  BASE_DIR = "https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/"
else:
  BASE_DIR = ".."


Cargamos los datasets de cada uno de los indicadores a predecir. Los mismos pueden descargarse del siguiente [link](https://drive.google.com/drive/folders/1NVxgmk_pED1FtnqWl_M4-2vRPiyd6Pri?usp=sharing).

In [ ]:
#Importacion de los dataset por separado

education =  pd.read_csv(BASE_DIR + '/educacion_mercosur_cleaned.csv')
gender = pd.read_csv(BASE_DIR + 'gender_mercosur_cleaned.csv')
environment = pd.read_csv(BASE_DIR + 'environment_mercosur_cleaned.csv')
health = pd.read_csv(BASE_DIR + 'health_mercosur_cleaned.csv')
poverty = pd.read_csv(BASE_DIR + 'poverty_mercosur_cleaned.csv')
social_protection = pd.read_csv(BASE_DIR + 'socialprotection_mercosur_cleaned.csv')

In [ ]:
#Corrección del nombre de la primera columna

education = education.rename(columns={"Unnamed: 0": "Features"})
gender = gender.rename(columns={"Unnamed: 0": "Features"})
environment = environment.rename(columns={"Unnamed: 0": "Features"})
health = health.rename(columns={"Unnamed: 0": "Features"})
poverty = poverty.rename(columns={"Unnamed: 0": "Features"})
social_protection = social_protection.rename(columns={"Unnamed: 0": "Features"})

Cargamos ahora la metadata de cada una de las variables para cada indicador. La misma puede descargarse desde el siguiente [link](https://drive.google.com/drive/folders/11QPQHD_hxeMZZ-VH9ppM9KXL46HRxSjy?usp=sharing).

La metadata es una explicación breve sobre el significado de cada variable y la fuente de datos, en criollo podríamos decir que básicamente es un "Diccionario de datos". Estos datos deben ser tomados de consulta y para dar sentido a los valores o información hallada.

In [ ]:
#Importacion de la metadata

metadata_education = pd.read_csv(BASE_DIR + 'metadata_education.csv')
metadata_gender = pd.read_csv(BASE_DIR + 'metadata_gender.csv')
metadata_environment = pd.read_csv( BASE_DIR + 'metadata_environment.csv')
metadata_health = pd.read_csv(BASE_DIR + 'metadata_health.csv')
metadata_poverty = pd.read_csv(BASE_DIR + 'metadata_poverty.csv')
metadata_socialprotection = pd.read_csv(BASE_DIR + 'metadata_socialprotection.csv')

# Limpieza y Curación de Datos

## SALUD

## GÉNERO

## Parte 1: Limpieza

En primer lugar, construimos un dataset exclusivo de Argentina.

In [ ]:
argentina_gender = gender[gender["Country Code"]== "ARG"]

Se eliminan todos los datos previos al año 1990 (es decir, la franja temporal desde 1970 hasta 1989).

In [ ]:
# Reescribimos el nombre de las columnas con años
argentina_gender = argentina_gender.drop(["Indicator", "Country Name", "Country Code","Indicator Code"], axis=1)
argentina_gender = argentina_gender.rename(columns = lambda x : str(x)[:4])
argentina_gender = argentina_gender.rename(columns = {"Feat": "Features"})

#Eliminamos la información de los años no relevante para el análisis
years_to_drop = pd.period_range(start='1970-01-01', end='1989-01-01', freq="Y").astype(str)
argentina_gender = argentina_gender.drop(years_to_drop , axis=1)
argentina_gender = argentina_gender.drop(["2019"] , axis=1)

argentina_gender

Se filtran las 15 features seleccionadas en el proceso de análisis y visualización.

In [ ]:
# Listado de ID Code seleccionados originalmente:
GenderIDCodeOriginal = pd.Index(['SP.DYN.TO65.MA.ZS', 'SP.DYN.TO65.FE.ZS', 'SP.DYN.TFRT.IN',
       'SP.DYN.LE00.MA.IN', 'SP.DYN.LE00.FE.IN', 'SP.DYN.AMRT.MA',
       'SP.DYN.AMRT.FE', 'SP.ADO.TFRT', 'SL.TLF.TOTL.FE.ZS',
       'SL.TLF.CACT.MA.ZS', 'SL.TLF.CACT.FE.ZS', 'SL.EMP.WORK.MA.ZS',
       'SL.EMP.WORK.FE.ZS', 'SL.EMP.VULN.MA.ZS', 'SL.EMP.VULN.FE.ZS'])

GenderIDCodeOriginal


Se considera necesario realizar salvedades de los features seleccionados en el análisis anterior:

- Dado que en la selección realizada, la feature **Población activa, mujeres (% de la población activa total)** no existe para varones, se decide reemplazar dicha feature. Por la siguiente feature: **Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)**.

- Además, dado que las features de: **tasa de mortalidad (por sexo)**, **Supervivencia hasta los 65 años (por sexo)** y  **Esperanza de vida al nacer (por sexo)**, son features que procuran medir/representar la dimension salud y demográfica del país por género, se decide retener solo una de 3 features y reemplazar las otras dos por features que representen otra dimensión social.


Así, se utilizan para realizar el reemplazo, las siguientes features:   **Proporción de inscripciones de mujeres con respecto a varones en la educación secundaria (%)**,  **Proporción de niñas con respecto a niños en educación primaria y secundaria (%)** y **Inscripción escolar, nivel terciario, varones (% bruto)** y **Inscripción escolar, nivel terciario, mujeres (% bruto)**. Y se retiene solo la feature de esperanza de vida al nacer por sexo.


In [ ]:
# Agregamos los dos ID de los features mencionados
GenderMap = gender[["Indicator Code", "Features"]].drop_duplicates()
add_features = ["Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)", "Proporción de inscripciones de mujeres con respecto a varones en la educación secundaria (%)" , "Proporción de niñas con respecto a niños en educación primaria y secundaria (%)", "Inscripción escolar, nivel terciario, varones (% bruto)", "Inscripción escolar, nivel terciario, mujeres (% bruto)" ]
NewIDCode = GenderMap[GenderMap["Features"].isin(add_features)]["Indicator Code"]
GenderIDCode = GenderIDCodeOriginal.to_list() + NewIDCode.to_list()
len(GenderIDCode)

In [ ]:
#Removemos los ID correspondientes
id_to_remove = {"SL.TLF.TOTL.FE.ZS", "SP.DYN.TO65.MA.ZS", "SP.DYN.TO65.FE.ZS", "SP.DYN.AMRT.MA", "SP.DYN.AMRT.FE" }
GenderIDCode = [id for id in GenderIDCode if id not in id_to_remove] 

print("Luego de incorporar los nuevos ID y de eliminar aquellos que se están reemplazando, \n la lista contiene los siguientes " +  str(len(GenderIDCode)) + " ID:")

GenderIDCode

In [ ]:
# Se realiza un mapeo para vincular el ID Code seleccionado con el feature que corresponde.
GenderMap = gender[["Indicator Code", "Features"]].drop_duplicates()

# Se retienen aquellos Fatures Seleccionados
GenderFeatures = GenderMap[GenderMap["Indicator Code"].isin(GenderIDCode)]
print ("El listado de los " +  str(len(GenderFeatures)) + " features es el siguiente: \n")

GenderFeatures = GenderFeatures["Features"].to_list()

GenderFeatures

Análisis del significado de las variables seleccionadas considerando la metadata.

In [ ]:
# Visualización del metadata para de los features seleccionados

metadata_gender[metadata_gender["INDICATOR_CODE"].isin(GenderIDCode)].drop(columns = [ "INDICATOR_CODE", "Unnamed: 0"], axis=1).set_index("INDICATOR_NAME")

Los 15 features seleccionados se consideran representativos de diferentes dimensiones sociales que se ven atravesadas por inequidades de género en el país.

Se procuró abordar la dimensión salud - demográfica, educativa, política y económica.

Dentro de la **dimensión salud y demográfica** pueden reconocerse los siguientes features: esperanza de vida al nacer y tasa de fertilidad.

Como representantes de la **dimensión política** se considera la proporción de mujures que componen las cámaras del poder legislativo a nivel nacional.

Por su parte, respecto a la **dimensión educativa**, se considera que los features de proporción de niñas respecto a los niños en educación primaria, secundaria y terciaria son representativos de la diferencia del acceso a la educación por sexo.

Por último, todos los features vinculados al trabajo remunerado, trabajo informal, población economicamente activa, se consideran como representativos de la situación de **empleo y actividad económica** de argentina .




## Parte 2: Análisis y Curación

#### 1 Detección y análisis de valores nulos

In [ ]:
#Creamos Dataset de los Features seleccionados para corroborar la cantidad de nans que existen
GenderFeatures_dataset = argentina_gender[argentina_gender["Features"].isin(GenderFeatures)]
# Agrupamos
GenderFeatures_dataset_nan = GenderFeatures_dataset.groupby(["Features"]).count()
# Creamos una variable que mida el porcentaje de registros
GenderFeatures_dataset_nan["Porcentaje de Registros"] = (GenderFeatures_dataset_nan.sum(axis=1)/29)*100
# Ordenamos de mayor a menor pje de registros
GenderFeatures_dataset_nan["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=False)



Como puede observarse, de las 15 features seleccionadas, solo 3 tiene un porcentaje de registro menor al 75%. El resto tienen alto nivel de información. Con el propósito de observar la distribución de los valores nulos para cada feature, se grafica un mapa de calor donde se puede evidenciar en qué años existen valores faltantes.

In [ ]:
#Graficamos Mapa de Calor
plt.figure(figsize=(20, 10))
sns.heatmap(GenderFeatures_dataset_nan.drop(["Porcentaje de Registros"], axis=1), vmin=0, vmax=1, cbar=True, linewidths=.3, cmap="PRGn")
plt.yticks(fontsize=16)
plt.xticks(fontsize=16, rotation=90)
plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en Argentina")

Como puede observarse en el mapa de calor, para la varios features la información es completa.

Todas aquellas features vinculadas a la **salud**, poseen información *completa*.

En el caso de los features referidos al **empleo** (empleo vulnerable y trabajadores asalariados, por sexo) *solo falta información para el año 1990*.

Mientras que los features con *menor información* son los referidos a la dimensión **educativa** y **política**.


Con el propósito de determinar el motivo de los valores faltantes, se corrobora la información disponible para los años anteriores a 1990 de los features seleccionados:

In [ ]:
# Construimos dataset con todos los años
argentina_allyears = gender[gender["Country Code"]== "ARG"].drop(["Indicator", "Country Name", "Country Code","Indicator Code"], axis=1)
argentina_allyears = argentina_allyears.rename(columns = lambda x : str(x)[:4])
argentina_allyears = argentina_allyears.rename(columns = {"Feat": "Features"})
argentina_allyears = argentina_allyears[argentina_allyears["Features"].isin(GenderFeatures)].groupby(["Features"]).count()

#Graficamos Mapa de Calor
plt.figure(figsize=(20, 10))
sns.heatmap(argentina_allyears.drop(["2019"] , axis=1), vmin=0, vmax=1, cbar=True, linewidths=.3, cmap="PRGn")
plt.yticks(fontsize=16)
plt.xticks(fontsize=16, rotation=90)
plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en Argentina")


Como se observa en el mapa de calor que considera todos los años del dataset orginal:
- Las features de **empleo vulnerable** y de **trabajadores asalariados** empieza a tener información disponible recién a partir de 1991. Cabe destacar que durante el período 1970 a 1983 el país vivió bajo dictadura militar, exceptuando los años de 1973 a 1976; lo que implica una fuerte inestabilidad en la recolección y generación de datos. Además, luego de la vuelta a la democracia el país se encontraba en un contexto de crisis social que finalizó en una gran crisis económica con la hiperinflación de 1989. El desorden económico era tal que dificiltaba la medición de las diferentes variables económicas. Con el comienzo de la convertibilidad, se reestableció (aunque de manera momentánea) el orden de las cuentas nacionales y se incorporó la medidición de diferentes indicadores. Es por ello que recién a partir de 1991 existe información para dichas features.
- Por su parte, **la tasa de población activa** por sexo, coincide en la falta de datos durante todo el período anterior a 1989 pero, a diferencia de las anteriores, tiene su primer regisitro en 1990. Mostrando información completa para proceder con el análisis.
- Mientras que, las features de **inscripción escolar nivel terciario, por sexo**, muestran información para todo el período de 1970 a 1987 y 1999 a 2017 pero se observa una discontinuidad en el relevamiento de los datos de 1988 a 1998. En este caso deberán completarse los nans con la metodología que se considere mas adecuada. Cabe destacar que implica que dentro del período relevente de análisis, existen 9 años sin información (1990 a 1998).
- En cuanto a las features de **proporción de inscripciones de mujeres con respecto a varones en la educación secundaria (%)** y **proporción de niñas con respecto a niños en educación primaria y secundaria (%)**, la falta de información es similar a la del feature anteriormente mencionado. Existe información para todo el período (1970-2018) exceputando los años de 1989 a 1995. Lo que implica que será necesario completar datos para los primeros 7 años del período de análisis.
- Por último, respecto a la **Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)** puede observarse que, si se considera todo el período (1970-2018), es la feature con menor proporción de datos. El primer año con información es 1990, pero luego se da una discontinuidad de datos por un período de 6 años (1991-1996). Sin embargo, la serie para los años siguientes se encuentra con gran proporción de informacióne exceptuando los años: 1999, 2001, 2013 y 2015. Vale destacar que, tanto en 1999 como 2001, en el país se estaba viviendo una de las mayores crisis politicas de la historia argentina, implicando un gran inestabilidad en quienes se encontraban en el poder, dificilutando la medición confiable de la información. 

#### 2 Cálculo de estadísticos y distribuciones

In [ ]:
#Se transforma el dataset

GenderDataMelt = pd.melt(GenderFeatures_dataset, id_vars = ["Features"])
GenderDataMelt["variable"] = pd.to_numeric(GenderDataMelt["variable"] )
GenderDataMelt = GenderDataMelt.rename(columns = {"variable": "Year"})


In [ ]:
# Calculamos estadisticos por feature para la totalidad de los años
GenderDataMelt.drop("Year", axis=1).groupby(["Features"]).describe()

Dado que la dimensión de análisis es el género, siempre que sea posible, se realizaran los boxplot y las series de tiempo de las features agrupadas por sexo para poder evidenciar la diferencia en los comportamientos de cada features según sexo.

In [ ]:
# Features que estian discriminados por sexo

features_per_gender_to_remove = {'Esperanza de vida al nacer, varones (años)',
 'Esperanza de vida al nacer, mujeres (años)',
 'Tasa de población activa, varones (% de la población masculina mayor de 15 años) (estimación modelado OIT)',
 'Tasa de población activa, mujeres (% de la población femenina mayor de 15 años) (estimación modelado OIT)',
 'Trabajadores asalariados (empleados), varones (% del empleo masculino)',
 'Trabajadores asalariados (empleados), mujeres (% del empleo femenino)',
 'Empleo vulnerable, varones (% del empleo masculino)',
 'Empleo vulnerable, mujeres (% del empleo femenino)',
 'Inscripción escolar, nivel terciario, varones (% bruto)',
 'Inscripción escolar, nivel terciario, mujeres (% bruto)' }
#Armamos una lista con aquellos que no se están discriminados por sexo
resto_gender_features = [id for id in GenderFeatures if id not in features_per_gender_to_remove] 
resto_gender_features

Boxplot de aquellos features que pueden graficarse por sexo:

In [ ]:
_condicion = ((GenderDataMelt["Features"]=="Trabajadores asalariados (empleados), mujeres (% del empleo femenino)") | (GenderDataMelt["Features"]=="Trabajadores asalariados (empleados), varones (% del empleo masculino)"
))
plt.figure(figsize=(10,7))
plt.figure(figsize=(10,6))
sns.boxplot(data=GenderDataMelt[_condicion], x="Features", y="value", width=0.1, hue = "Features")
plt.title("Boxplot de Trabajadores Asalariados por sexo" , fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Valor', fontsize=12)
plt.xlabel('', fontsize=12)


Como puede observarse en el boxplot y en tabla con las estadísiticas descriptivas de cada feature, los features de  **Trabajadores asalariados (empleados), mujeres (% del empleo femenino)**  y **Trabajadores asalariados (empleados), varones (% del empleo masculino)**  tienen una media de 77.575893 y 70.999929, respectivamente. Esto implica que en el período de análisis, de todas las mujeres que trabajan en promedio el 77.57% lo hacen en condición de asalariadas, mientras que para el caso de los hombres el promedio es más bajo 71%. Sin embargo, la desviación estándar del feature para mujeres es mayor que el de los hombres.

In [ ]:
_mujeres = "Trabajadores asalariados (empleados), mujeres (% del empleo femenino)"
_desviacion_standar_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].std()
_media_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].mean()
coef_var_m = _desviacion_standar_m/_media_m

_hombres = "Trabajadores asalariados (empleados), varones (% del empleo masculino)"
_desviacion_standar_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].std()
_media_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].mean()
coef_var_h = _desviacion_standar_h/_media_h

print( "El coeficiente de variación de la serie referida a las mujeres es de " + str(coef_var_m) + " ,\n mientras que la de los hombres es de " + str(coef_var_h))



Además, si se comparan las dos series, las mismas  tienen un coeficiente de variación similar, implicando que los datos tienen casi el mismo grado de heterogeneidad, aunque el de las mujeres es levemente mayor.

In [ ]:
_condicion = ((GenderDataMelt["Features"]=="Empleo vulnerable, mujeres (% del empleo femenino)") | (GenderDataMelt["Features"]=="Empleo vulnerable, varones (% del empleo masculino)"
))
plt.figure(figsize=(10,7))
plt.figure(figsize=(10,6))
sns.boxplot(data=GenderDataMelt[_condicion], x="Features", y="value", width=0.1, hue = "Features")
plt.title("Boxplot de Empleo Vulnerable por sexo", fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Valor', fontsize=12)


Como puede observarse en el boxplot y en tabla con las estadísiticas descriptivas de cada feature, los features del **empleo vulnerable mujeres (% del empleo femenino)** y **empleo vulnerable varones (% del empleo masculino)** tienen una media de 19.018501y 23.487000, respectivamente. Esto implica que en el período de análisis, de todas las mujeres que trabajan en promedio el 20% lo hacen en condiciones vulnerables, mientras que para el caso de los hombres el promedio es más mayor 23.48%.

Vale destacar que, dado que el feature en análisis considera el % de trabajadores familiares no remunerados y a los trabajadores autónomos como porcentaje del empleo total, según corresponda, excluye del análisis las personas que se encuentran trabajando en el sector informal, así como también la trabajo no remunerado realizado dentro del hogar.
Por eso, tiene sentido que sea mayor en hombres que en mujeres.

In [ ]:
_mujeres = "Empleo vulnerable, mujeres (% del empleo femenino)"
_desviacion_standar_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].std()
_media_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].mean()
coef_var_m = _desviacion_standar_m/_media_m

_hombres = "Empleo vulnerable, varones (% del empleo masculino)"
_desviacion_standar_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].std()
_media_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].mean()
coef_var_h = _desviacion_standar_h/_media_h

print( "El coeficiente de variación de la serie referida a las mujeres es de " + str(coef_var_m) + " ,\n mientras que la de los hombres es de " + str(coef_var_h))



En este caso, se evidencia que la serie de las mujeres es mas heterogénea que la de los hombres, implicando una mayor dispersión en los datos que en el caso de los varones.

In [ ]:
_condicion = ((GenderDataMelt["Features"]=="Esperanza de vida al nacer, varones (años)") | (GenderDataMelt["Features"]=="Esperanza de vida al nacer, mujeres (años)"))
plt.figure(figsize=(10,7))
plt.figure(figsize=(10,6))
sns.boxplot(data=GenderDataMelt[_condicion], x="Features", y="value", width=0.1, hue = "Features")
plt.title("Boxplot de Esperanza de Vida al Nacer por sexo", fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Valor', fontsize=12)
plt.xlabel('', fontsize=12)

Como puede observarse en el boxplot y en tabla con las estadísiticas descriptivas de cada feature, los features del **Esperanza de vida al nacer, mujeres (años)** y **Esperanza de vida al nacer, varones (años)** tienen una media de 77.609103 y 70.761862, respectivamente. Esto implica que , en promedio, las mujeres tienen una esperanza de vida mayor a la de los hombres.

In [ ]:
_mujeres = "Esperanza de vida al nacer, mujeres (años)"
_desviacion_standar_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].std()
_media_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].mean()
coef_var_m = _desviacion_standar_m/_media_m

_hombres = "Esperanza de vida al nacer, varones (años)"
_desviacion_standar_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].std()
_media_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].mean()
coef_var_h = _desviacion_standar_h/_media_h

print( "El coeficiente de variación de la serie referida a las mujeres es de " + str(coef_var_m) + " ,\n mientras que la de los hombres es de " + str(coef_var_h))


En este caso, el coeficiente de variación para ambos sexos es similar. Es decir, que en todo el período de análisis, las mujeres mantienen una mayor esperanza de vida que los hombres, dado que las series son bastente homogeneas.

In [ ]:
_condicion = ((GenderDataMelt["Features"]=='Inscripción escolar, nivel terciario, varones (% bruto)') | (GenderDataMelt["Features"]=='Inscripción escolar, nivel terciario, mujeres (% bruto)'))
plt.figure(figsize=(10,7))
plt.figure(figsize=(10,6))
sns.boxplot(data=GenderDataMelt[_condicion], x="Features", y="value", width=0.1, hue = "Features")
plt.title("Boxplot de Insripción Escolar (Terciario) por sexo", fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Valor', fontsize=12)
plt.xlabel('', fontsize=12)

Como puede observarse en el boxplot y en tabla con las estadísiticas descriptivas de cada feature, los features del **Inscripción escolar, nivel terciario, mujeres (% bruto)** y **Inscripción escolar, nivel terciario, varones (% bruto)** tienen una media de 85.494711 y 55.613296, respectivamente. Esto implica que , en promedio, las mujeres tienen tasa de participación en la educación terciaria que los hombres.

In [ ]:
_mujeres = "Inscripción escolar, nivel terciario, mujeres (% bruto)"
_desviacion_standar_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].std()
_media_m = GenderDataMelt[GenderDataMelt["Features"]==_mujeres]["value"].mean()
coef_var_m = _desviacion_standar_m/_media_m

_hombres = "Inscripción escolar, nivel terciario, varones (% bruto)"
_desviacion_standar_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].std()
_media_h = GenderDataMelt[GenderDataMelt["Features"]==_hombres]["value"].mean()
coef_var_h = _desviacion_standar_h/_media_h


print( "El coeficiente de variación de la serie referida a las mujeres es de " + str(coef_var_m) + " ,\n mientras que la de los hombres es de " + str(coef_var_h))


Además, si se comparan las dos series, las mismas  tienen un coeficiente de variación similar, implicando que los datos tienen casi el mismo grado de heterogeneidad, aunque el de las mujeres es levemente mayor.

Boxplot del resto de los features:

In [ ]:

GenderDataMelt = GenderDataMelt.copy()

for features in resto_gender_features:
    plt.figure(figsize=(10,6))
    sns.boxplot(data=GenderDataMelt[GenderDataMelt["Features"]==features], x="Features", y="value", width=0.1)
    plt.title("Boxplot de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.ylabel('Valor', fontsize=12)
    plt.xlabel('', fontsize=12)


Series de tiempo agrupadas por sexo:

In [ ]:
_condicion = ((GenderDataMelt["Features"]=="Trabajadores asalariados (empleados), mujeres (% del empleo femenino)") | (GenderDataMelt["Features"]=="Trabajadores asalariados (empleados), varones (% del empleo masculino)"
))
plt.figure(figsize=(10,7))
sns.pointplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Features")
plt.title("Serie Temporal del % de Trabajadores Asalariados \n Hombres y Mujeres", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:
  
_condicion = ((GenderDataMelt["Features"]=="Empleo vulnerable, mujeres (% del empleo femenino)") | (GenderDataMelt["Features"]=="Empleo vulnerable, varones (% del empleo masculino)"
))
plt.figure(figsize=(10,7))
sns.pointplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Features")
plt.title("Serie Temporal del % de Empleo Vulnerable \n Hombres y Mujeres", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:
_condicion = ((GenderDataMelt["Features"]=="Esperanza de vida al nacer, varones (años)") | (GenderDataMelt["Features"]=="Esperanza de vida al nacer, mujeres (años)"))
plt.figure(figsize=(10,7))
sns.pointplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Features")
plt.title("Serie Temporal de la Esperanza de vida al nacer \n Hombres y Mujeres", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:

_condicion = ((GenderDataMelt["Features"]=="Tasa de población activa, varones (% de la población masculina mayor de 15 años) (estimación modelado OIT)") | (GenderDataMelt["Features"]=="Tasa de población activa, mujeres (% de la población femenina mayor de 15 años) (estimación modelado OIT)"))
plt.figure(figsize=(10,7))
sns.pointplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Features")
plt.title("Serie Temporal de la Tasa de Población Activa \n Hombres y Mujeres", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:

_condicion = ((GenderDataMelt["Features"]=='Inscripción escolar, nivel terciario, varones (% bruto)') | (GenderDataMelt["Features"]=='Inscripción escolar, nivel terciario, mujeres (% bruto)'))
plt.figure(figsize=(10,7))
sns.pointplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Features")
plt.title("Serie Temporal de Inscripción Escolar Terciario \n Hombres y Mujeres", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)


Serie de tiempo de los features restantes:

In [ ]:
for features in resto_gender_features:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=GenderDataMelt[GenderDataMelt["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

#### 3 Análisis de posible imputación de valores faltantes

En el caso de **Género**, cada feature muestra sus particularidades. Es por eso que se procede a completar la información faltante en base a diferentes criterios según corresponda.


In [ ]:
#### PRIMER CRITERIO DE IMPUTACIÓN DE DATOS
# Seleccionamos features con NaNs en el 2018
Gender_DATASET_step_one = GenderFeatures_dataset.copy()
GenderFeatures_dataset_null_2018 = Gender_DATASET_step_one[Gender_DATASET_step_one["2018"].isnull()]
gender_features_null_2018 = GenderFeatures_dataset_null_2018["Features"].tolist()
GenderFeatures_dataset_2018_null = Gender_DATASET_step_one[Gender_DATASET_step_one["Features"].isin(gender_features_null_2018)]
GenderFeatures_dataset_2018_not_null = Gender_DATASET_step_one[~Gender_DATASET_step_one["Features"].isin(gender_features_null_2018)]
# Seleccionamos solo los años que usamos para corroborar si existencia tendencia:
Gender_DATASET_check = GenderFeatures_dataset_2018_null[["Features", "2014","2015", "2016", "2017", "2018"]]

# En caso que exista una tendencia creciente o decreciente, aplicamos la tasa de crecimiento promedio de los ultimos tres años para interpolar el dato faltante.
IncreasingTrend_Gender = ((Gender_DATASET_check['2014'] - Gender_DATASET_check['2015']) > 0) & ((Gender_DATASET_check['2015'] - Gender_DATASET_check['2016'])>0) & ((Gender_DATASET_check['2016'] - Gender_DATASET_check['2017'])>0)
DecreasingTrend_Gender = ((Gender_DATASET_check['2014'] - Gender_DATASET_check['2015']) < 0) & ((Gender_DATASET_check['2015'] - Gender_DATASET_check['2016'])<0) & ((Gender_DATASET_check['2016'] - Gender_DATASET_check['2017'])<0)

# Features con tendencia creciente:
print("Los features sin datos para 2018, que muestran una tendencia creciente son: " +  str(Gender_DATASET_check[IncreasingTrend_Gender]["Features"].tolist()) )
print("Los features sin datos para 2018, que muestran una tendencia decreciente son: " +  str(Gender_DATASET_check[DecreasingTrend_Gender]["Features"].tolist()) )


Para aquellos features que muestran tendencia (ya sea creciente o decreciente) se completará el vacío del año **2018** a través de la aplicación de la **tasa de variación promedio de los últimos 3 años al valor de 2017**.

In [ ]:
# Se calcula la tasa de variación promedio

GenderFeatures_dataset_2018_null["Var. First Year"] = (GenderFeatures_dataset_2018_null['2014'] - GenderFeatures_dataset_2018_null['2015'])/(GenderFeatures_dataset_2018_null['2015'])
GenderFeatures_dataset_2018_null["Var. Second Year"] = (GenderFeatures_dataset_2018_null['2015'] - GenderFeatures_dataset_2018_null['2016'])/(GenderFeatures_dataset_2018_null['2016'])
GenderFeatures_dataset_2018_null["Var. Third Year"] = (GenderFeatures_dataset_2018_null["2016"] - GenderFeatures_dataset_2018_null["2017"])/(GenderFeatures_dataset_2018_null["2017"])
GenderFeatures_dataset_2018_null["Growth Rate"] = (GenderFeatures_dataset_2018_null["Var. First Year"]+GenderFeatures_dataset_2018_null["Var. Second Year"]+GenderFeatures_dataset_2018_null["Var. Third Year"])/3

#Completamos dataset
GenderFeatures_dataset_2018_null["2018"] = GenderFeatures_dataset_2018_null["2018"].fillna(GenderFeatures_dataset_2018_null["2017"] * (1 + GenderFeatures_dataset_2018_null["Growth Rate"]))
GenderFeatures_dataset_2018_null_fill = GenderFeatures_dataset_2018_null.copy()
GenderFeatures_dataset_2018_null_fill = GenderFeatures_dataset_2018_null_fill.drop(["Var. First Year", "Var. Second Year", "Var. Third Year", "Growth Rate"], axis=1)

# Dataset completo
Gender_DATASET_step_two =  pd.concat([GenderFeatures_dataset_2018_null_fill,GenderFeatures_dataset_2018_not_null], ignore_index=False)

# Check
Gender_DATASET_step_two["2018"].isnull().sum()



Como segundo criterio, se completan los datos de **1990 replicando el valor de 1991**. Este criterio se utilizará para completar el valor de aquellas features relativas a la dimensión socio-económica, dado que el primer valor las mismas en todo el período (1970-2018) es 1991.

In [ ]:
#### SEGUNDO CRITERIO DE IMPUTACIÓN DE DATOS
# Seleccionamos features con NaNs en el 1990
Gender_DATASET_step_two = Gender_DATASET_step_two.copy()
gender_features_fill_1990_step_two = ["Trabajadores asalariados (empleados), varones (% del empleo masculino)", "Empleo vulnerable, mujeres (% del empleo femenino)", "Empleo vulnerable, varones (% del empleo masculino)", "Trabajadores asalariados (empleados), mujeres (% del empleo femenino)" ] 
GenderFeatures_dataset_1990_null = Gender_DATASET_step_two[Gender_DATASET_step_two["Features"].isin(gender_features_fill_1990_step_two)]
#En este caso a pesar que se define a la variable como not null, existen features que tienen null en 1990 pero no se utiliza el mismo criterio para completar.
GenderFeatures_dataset_1990_not_null = Gender_DATASET_step_two[~Gender_DATASET_step_two["Features"].isin(gender_features_fill_1990_step_two)]

# Completamos los valores nulos de 1990 con los de 1991.
GenderFeatures_dataset_1990_null["1990"] = GenderFeatures_dataset_1990_null["1990"].fillna(GenderFeatures_dataset_1990_null["1991"])

# Chequeamos que no existan features con nans para 1990
GenderFeatures_dataset_1990_null["1990"].isnull().sum()

# Dataset completo
Gender_DATASET_step_three =  pd.concat([GenderFeatures_dataset_1990_null,GenderFeatures_dataset_1990_not_null], ignore_index=False)


Por su parte, para terminar de completar la información faltante en **los features relativos a educación**, se considera un tercer criterio. Dado que, como se observó en el heatmap, de todo el período (1970-2018) estos features tienen información para años anteriores a 1990, se procede a graficar las series de tiempo para todo el período con el propósito de observar el comportamiento de la serie y determinar si el pasado sirve para interpolar los años faltantes. Además, se considera el feature de **Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)** tambien en el dataset de todo el período, dado que, a pesar de que no existe información regsitra para los años anteriores a 1990, se le aplicará la metodología de interpolación.

In [ ]:
#### TERCER CRITERIO DE IMPUTACIÓN DE DATOS
# Seleccionamos features relativos a educación con NaNs 
Gender_DATASET_step_three = Gender_DATASET_step_three.copy()
gender_features_fill_step_three = ["Proporción de niñas con respecto a niños en educación primaria y secundaria (%)", "Inscripción escolar, nivel terciario, varones (% bruto)", "Inscripción escolar, nivel terciario, mujeres (% bruto)", "Proporción de inscripciones de mujeres con respecto a varones en la educación secundaria (%)", "Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)" ]
 
# Creamos dataset considerando todo el periodo
argentina_allperiod = gender[gender["Country Code"]== "ARG"].drop(["Indicator", "Country Name", "Country Code","Indicator Code"], axis=1)
argentina_allperiod = argentina_allperiod[argentina_allperiod["Features"].isin(gender_features_fill_step_three)]
GenderDataMelt_Allyears = pd.melt( argentina_allperiod, id_vars = ["Features"])
GenderDataMelt_Allyears["variable"] = pd.to_numeric(GenderDataMelt_Allyears["variable"] )
GenderDataMelt_Allyears = GenderDataMelt_Allyears.rename(columns = {"variable": "Year"})
GenderDataMelt_Allyears["Year"] = GenderDataMelt_Allyears["Year"].astype(int)

In [ ]:
#Se grafica para determinar si es posible considerar la informacion de los años anteriores.

for features in gender_features_fill_step_three:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=GenderDataMelt_Allyears[GenderDataMelt_Allyears["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=12, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('', fontsize=12)
    plt.ylabel('Valor', fontsize=16)

A partir de los gráficos, se observa que en algunos casos es posible realizar una interpolación. En caso de los features de **proporción de inscripciones mujeres respecto a la cantidad de hombres inscriptos en educación secundaria (%)** y **proporción de niñas respecto a niños en educación primaria y secundaria (%)** a pesar de que se observa una serie muy irregular, la varianza de ambas es muy baja. Es decir, el rango de los valores que asumen los features es pequeño, es por eso que se procede a utilizar una **interpolación lineal**. Por su parte, para completar los valores de los features de **Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)** e **Inscripción escolar, nivel terciario, mujeres (% bruto)** también se considera óptima la aplicación de interpolación.


In [ ]:
## APLICACIÓN DEL TERCER CRITERIO


GenderDataMelt_Allyears_int = GenderDataMelt_Allyears.copy()
GenderDataMelt_Allyears_int['Year'] = pd.to_datetime(GenderDataMelt_Allyears_int['Year'], format='%Y').dt.year
GenderDataMelt_Allyears_int = GenderDataMelt_Allyears_int.sort_values(by = ["Features", "Year"]).interpolate()

In [ ]:
# Para unir con la nueva información, transformamos le último dataset que tenemos.
GenderDataMelt_step_three = pd.melt( Gender_DATASET_step_three, id_vars = ["Features"])
GenderDataMelt_step_three["variable"] = pd.to_numeric(GenderDataMelt_step_three["variable"] )
GenderDataMelt_step_three = GenderDataMelt_step_three.rename(columns = {"variable": "Year"})
GenderDataMelt_step_three["Year"] = GenderDataMelt_step_three["Year"].astype(int)
GenderDataMelt_step_three = GenderDataMelt_step_three

#Solo reemplazmos los datos de los dos features que son relevantes
gender_features_fill_step_three = ["Proporción de niñas con respecto a niños en educación primaria y secundaria (%)", "Proporción de inscripciones de mujeres con respecto a varones en la educación secundaria (%)",  "Proporción de escaños ocupados por mujeres en los parlamentos nacionales (%)", "Inscripción escolar, nivel terciario, mujeres (% bruto)"]
# Reestructuramos dataset
GenderDataMelt_after_applied_step_3 = GenderDataMelt_Allyears_int[GenderDataMelt_Allyears_int["Features"].isin(gender_features_fill_step_three)]
GenderDataMelt_after_applied_step_3 = GenderDataMelt_after_applied_step_3.rename(columns = {"value": "valor para completar"})
GenderDataMelt_step_three_aux = GenderDataMelt_step_three[GenderDataMelt_step_three["Features"].isin(gender_features_fill_step_three)].rename(columns = {"value": "valor original"})
GenderDataMelt_step_three_aux = GenderDataMelt_step_three_aux.set_index(["Features", "Year"])
GenderDataMelt_after_applied_step_3= GenderDataMelt_after_applied_step_3[(GenderDataMelt_after_applied_step_3["Year"]>=1990) | (GenderDataMelt_after_applied_step_3["Year"]>=2019)]
# Unimos dataset, completamos los nan
_df_3 = GenderDataMelt_after_applied_step_3.join(GenderDataMelt_step_three_aux, on = ["Features", "Year"], how="right")
_df_3["value"] = _df_3["valor original"].fillna(_df_3["valor para completar"])
_df_3 = _df_3.drop(["valor para completar", "valor original" ], axis=1)
# Resto de los features
_df_original = GenderDataMelt_step_three[~GenderDataMelt_step_three["Features"].isin(gender_features_fill_step_three)]
#Obtenemos ultimo dataset completo
GenderDataMelt_for_step_four = pd.concat([_df_original, _df_3 ])

Por último, se completa la información faltante del feature **Inscripción escolar, nivel terciario, varones (% bruto)** a traves de la consideración de los valores históricos, utilizando la metodología del **rolling**.


In [ ]:
## APLICACIÓN DEL CUARTO CRITERIO
GenderDataMelt_Allyears_rolling = GenderDataMelt_Allyears.copy()
GenderDataMelt_Allyears_rolling['Year'] = pd.to_datetime(GenderDataMelt_Allyears_rolling['Year'], format='%Y').dt.year
GenderDataMelt_Allyears_rolling.set_index("Year")
GenderDataMelt_Allyears_rolling = GenderDataMelt_Allyears_rolling.sort_values(by = ["Year","Features"])
# Se aplica un rolling de 12 períodos
gender_features_fill_step_four = ["Inscripción escolar, nivel terciario, varones (% bruto)"]
GenderDataMelt_Allyears_rolling_h = GenderDataMelt_Allyears_rolling[GenderDataMelt_Allyears_rolling["Features"].isin(gender_features_fill_step_four)]
GenderDataMelt_Allyears_rolling_h["valor para completar"] =  GenderDataMelt_Allyears_rolling_h["value"].rolling(12, min_periods=1).mean()
_filtro = GenderDataMelt_Allyears_rolling_h["Year"]>= 1990
GenderDataMelt_Allyears_rolling_h = GenderDataMelt_Allyears_rolling_h[_filtro].drop(["value"], axis=1)

In [ ]:
#Unificación de datasets
#Dataset con los 3 primeros criterios aplicados:
GenderDataMelt_for_step_four

#Solo reemplazmos los datos de los dos features que son relevantes
gender_features_fill_step_four

# Dataset con el criterio 4 aplicado
GenderDataMelt_Allyears_rolling_h.set_index(["Features", "Year"])

# Reestructuramos dataset
GenderDataMelt_step_four_aux = GenderDataMelt_for_step_four[GenderDataMelt_for_step_four["Features"].isin(gender_features_fill_step_four)].rename(columns = {"value": "valor original"})
GenderDataMelt_step_four_aux = GenderDataMelt_step_four_aux.set_index(["Features", "Year"])

# Unimos dataset, completamos los nan
_df_4 = GenderDataMelt_Allyears_rolling_h.join(GenderDataMelt_step_four_aux, on = ["Features", "Year"], how="right")
_df_4["value"] = _df_4["valor original"].fillna(_df_4["valor para completar"])
_df_4 = _df_4.drop(["valor para completar", "valor original" ], axis=1)

# Resto de los features
_df_original = GenderDataMelt_for_step_four[~GenderDataMelt_for_step_four["Features"].isin(gender_features_fill_step_four)]

#Obtenemos ultimo dataset completo
GenderFilledDATASET = pd.concat([_df_original, _df_4 ])


Realizamos gráfico de lineas para comprobar que las series se encuentren completas para todos los features.

In [ ]:
for features in GenderFeatures:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=GenderFilledDATASET[GenderFilledDATASET["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=12, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('', fontsize=12)
    plt.ylabel('Valor', fontsize=16)

FINAL FALTA

## POBREZA

## Parte 1: Limpieza

In [ ]:
argentina_poverty = poverty[poverty["Country Code"]== "ARG"]

In [ ]:
# Reescribimos el nombre de las columnas con años
argentina_poverty = argentina_poverty.drop(["Indicator", "Country Name", "Country Code","Indicator Code"], axis=1)
argentina_poverty = argentina_poverty.rename(columns = lambda x : str(x)[:4])
argentina_poverty = argentina_poverty.rename(columns = {"Feat": "Features"})

#Eliminamos la información de los años no relevante para el análisis
years_to_drop = pd.period_range(start='1970-01-01', end='1989-01-01', freq="Y").astype(str)
argentina_poverty = argentina_poverty.drop(years_to_drop , axis=1)
argentina_poverty = argentina_poverty.drop(["2019"] , axis=1)

argentina_poverty

In [ ]:
PovertyFeaturesOriginal = argentina_poverty["Features"].unique().tolist()
PovertyFeaturesOriginal

In [ ]:
# Visualización del metadata para de los features seleccionados

metadata_poverty[metadata_poverty["INDICATOR_NAME"].isin(PovertyFeaturesOriginal)].drop(columns = [ "INDICATOR_CODE", "Unnamed: 4", "Unnamed: 0"], axis=1).set_index("INDICATOR_NAME")

## Parte 2: Análisis y Curación

#### 1 Detección y análisis de valores nulos

In [ ]:
#Creamos Dataset de los Features seleccionados para corroborar la cantidad de nans que existen
PovertyFeatures_dataset = argentina_poverty[argentina_poverty["Features"].isin(PovertyFeaturesOriginal)]
# Agrupamos
PovertyFeatures_dataset_nan = PovertyFeatures_dataset.groupby(["Features"]).count()
# Creamos una variable que mida el porcentaje de registros
PovertyFeatures_dataset_nan["Porcentaje de Registros"] = (PovertyFeatures_dataset_nan.sum(axis=1)/29)*100
# Ordenamos de mayor a menor pje de registros
PovertyFeatures_dataset_nan["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=False)


Como puede observarse, de las 15 features seleccionadas, solo 1 tiene un porcentaje de registro realmente bajo. La información brindada por la **tasa de incidencia de la pobreza, sobre la linea de pobreza nacional**, no es suficiente para realizar el análisis, por eso la misma es **descartada**.

In [ ]:
# Se elimina la tasa de incidencia de la lista de features seleccionados:

features_to_remove = {"Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)"}
PovertyFeatures = [id for id in PovertyFeaturesOriginal if id not in features_to_remove]

#Creamos Dataset de los Features seleccionados para corroborar la cantidad de nans que existen
PovertyFeatures_dataset = argentina_poverty[argentina_poverty["Features"].isin(PovertyFeatures)]
# Agrupamos
PovertyFeatures_dataset_nan = PovertyFeatures_dataset.groupby(["Features"]).count()

In [ ]:
#Graficamos Mapa de Calor
plt.figure(figsize=(20, 10))
sns.heatmap(PovertyFeatures_dataset_nan, vmin=0, vmax=1, cbar=True, linewidths=.3, cmap="PRGn")
plt.yticks(fontsize=16)
plt.xticks(fontsize=16, rotation=90)
plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en Argentina")

Como puede observarse en el mapa de calor, descartando la feature de tasa de incidencia sobre la linea de pobreza, el resto de las features tiene casi información completa. En todos los casos falta información para el año 1990 y 2015. Respecto a la falta de datos para 2015, probablemente puede deberse a que no se consideraron los datos publicados por el INDEC dado que el mismo se encontraba intervenido.

#### 2 Cálculo de estadísticos y distribuciones

In [ ]:
#Se transforma el dataset

PovertyDataMelt = pd.melt(PovertyFeatures_dataset, id_vars = ["Features"])
PovertyDataMelt["variable"] = pd.to_numeric(PovertyDataMelt["variable"] )
PovertyDataMelt = PovertyDataMelt.rename(columns = {"variable": "Year"})


In [ ]:
# Calculamos estadisticos por feature para la totalidad de los años
PovertyDataMelt.drop("Year", axis=1).groupby(["Features"]).describe()

In [ ]:
PovertyDataMelt = PovertyDataMelt.copy()

for features in PovertyFeatures:
    plt.figure(figsize=(10,6))
    sns.boxplot(data=PovertyDataMelt[PovertyDataMelt["Features"]==features], x="Features", y="value", width=0.1)
    plt.title("Boxplot de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.ylabel('Valor', fontsize=12)
    plt.xlabel('', fontsize=12)

In [ ]:
PovertyDataMelt = PovertyDataMelt.copy()

for features in PovertyFeatures:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=PovertyDataMelt[PovertyDataMelt["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

#### 3 Análisis de posible imputación de valores faltantes

Como puede observarse en laos gráficos de líneas, para todos los features de pobreza es posible utilizar la interpolación lineal para completar la información del año 2015. Dado que para todos los casos se observa una continuidad en el comportamiento de la variable si se consideran los dos años anteriores a 2015 y los dos años siguientes.

En primer lugar, se completan los datos de **1990 replicando el valor de 1991**.

In [ ]:
#### PRIMER CRITERIO DE IMPUTACIÓN DE DATOS
# Seleccionamos features con NaNs en el 1990
PovertyFeatures_dataset_null = PovertyFeatures_dataset[PovertyFeatures_dataset["1990"].isnull()]
# Completamos los valores nulos de 1990 con los de 1991.
PovertyFeatures_dataset_null["1990"] = PovertyFeatures_dataset_null["1990"].fillna(PovertyFeatures_dataset_null["1991"])
# Redefinimos el dataset
PovertyFeatures_DATASET_step_one = PovertyFeatures_dataset_null


# Chequeamos que no existan features con nans para 1990
PovertyFeatures_DATASET_step_one["1990"].isnull().sum()

Se completan los datos para el año **2015**, utilizando la función **interpolate bajo el método lineal**.

In [ ]:
#### SEGUNDO CRITERIO DE IMPUTACIÓN DE DATOS ###

PovertyFeatures_DATASET_step_two = pd.melt(PovertyFeatures_DATASET_step_one, id_vars = ["Features"])
PovertyFeatures_DATASET_step_two["variable"] = pd.to_numeric(PovertyFeatures_DATASET_step_two["variable"] )
PovertyFeatures_DATASET_step_two = PovertyFeatures_DATASET_step_two.rename(columns = {"variable": "Year"})
PovertyFeatures_DATASET_step_two['Year'] = pd.to_datetime(PovertyFeatures_DATASET_step_two['Year'], format='%Y').dt.year
PovertyFeatures_DATASET_step_two = PovertyFeatures_DATASET_step_two.sort_values(by =["Features", "Year"]).interpolate()


# Check de que todos los años tienen datos
PovertyFeatures_DATASET_step_two.isnull().sum()


In [ ]:
Poverty_dataset_final = PovertyFeatures_DATASET_step_two
Poverty_dataset_final

In [ ]:
Poverty_dataset_final = Poverty_dataset_final.copy()

for features in PovertyFeatures:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=Poverty_dataset_final[Poverty_dataset_final["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=12)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

## EDUCACIÓN

### Parte 1: Limpieza

1. Eliminar todos los datos previos al año 1990 (es decir, la franja temporal desde 1970 hasta 1989).

In [ ]:
columns = education.columns
columnsDelete = columns[5:25].to_list()
columnsDelete = columnsDelete + [columns[-1]]

In [ ]:
RelevantYearsEducation = education.drop(columnsDelete, axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)

RelevantYearsEducation

2. Filtrar los datos anteriores considerando solo Argentina.

In [ ]:
RelevantYearsEducationArg = RelevantYearsEducation[RelevantYearsEducation["Country Name"]=="Argentina"]
RelevantYearsEducationArg

3. Restringir el conjunto de las variables del indicador a las 15 features elegidas (aquellas con más del 50% de valores no nulos) durante el desarrollo del Práctico anterior.

Primero se busca de nuevo la lista con los 15 features:

In [ ]:
EducationValuesByICode = education.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "Features"], axis=1)
EducationValuesByICode["Valor"] = EducationValuesByICode.sum(axis=1)
EducationValuesByICode = EducationValuesByICode.sort_values(by=['Valor'], ascending=False)
EducationValuesByICode = ((EducationValuesByICode/12)*100).drop(columns = ["Valor"])
EducationValuesByICode = EducationValuesByICode.rename(columns = lambda x : str(x)[:4])
round(EducationValuesByICode,2)

In [ ]:
EducationValuesByICode=EducationValuesByICode.reset_index()
EducationValuesByICode["Features"] = EducationValuesByICode["Indicator Code"]
EducationValuesByICode["Features"] = EducationValuesByICode["Features"].map(education.set_index("Indicator Code")["Features"].to_dict())
FeaturesEducationSelection = EducationValuesByICode.iloc[0:15][["Indicator Code", "Features"]].set_index("Indicator Code")
FeaturesEducationSelection

In [ ]:
EducationIDCode = FeaturesEducationSelection.reset_index()['Indicator Code'].to_list()

CleanInitialEducation = RelevantYearsEducationArg[RelevantYearsEducationArg["Indicator Code"].isin(EducationIDCode)]
CleanInitialEducation

4. Realizar una inspección acerca del significado de las variables valiendose de la información disponible en la metadata. Pueden guiarse de las siguientes preguntas:

- ¿Son todos los features representativos dentro del contexto argentino? ¿Existe alguno/s que podría no considerarse dado que no aplica o no aplicó a la realidad argentina? Justifique las respuestas.
- En las variables que involucran género y edad: ¿se consideran todos casos?. Por ejemplo: % de trabajadores asalariados hombres y % de trabajadoras asalariaradas mujeres. Si la respuesta es negativa, ¿cuál consideran que sería la decisión más acertada para abordar este aspecto?

In [ ]:
MetadataEducationRelevantFeatures = metadata_education[metadata_education["INDICATOR_CODE"].isin(EducationIDCode)].drop("Unnamed: 0", axis=1)
MetadataEducationRelevantFeatures

Puede apreciarse que los features son representativos dentro del contexto argentino dado que aplican a la realidad del sistema educativo en el país. No habria que eliminar ningun feature en este sentido.

Por otra parte, algunos features se refieren a determinadas edades, dejando fuera a ciertos rangos etarios. Esto se evidencia en los features llamados "Población entre 15 y 64 años de edad (% del total)" y "Población entre 0 y 14 años de edad (% del total)", dado que se excluye a la población mayor a 64 años. Sin embargo, no se dispone de un feature que contenga esta información en el dataset original, de manera que estamos trabajando con toda la información al respecto disponible en la base.

En relación a la consideración de un unico género, pareciera no haber inconvenientes.

- La información de la inscripción escolar del nivel primario, se ve implicada en dos features escogidos, uno para varones ("Inscripción escolar, nivel primario, varones (% bruto)") y otro para mujeres ("Inscripción escolar, nivel primario, mujeres (% bruto)"). De manera que se tienen en cuenta ambos generos.
- Un feature que puede generar confusión es el llamado "Educación de nivel primario, alumnos (% de mujeres)", dado que no se dispone del feature de % de hombres. Sin embargo, tal feature no se dispone en el dataset original. El feature que si se dispone es el de "Educación de nivel primario, alumnos", que contiene ambos géneros. De allí se desprende el % de mujeres que está contemplado en la lista de features relevantes.

### Parte 2: Análisis y Curación

#### 1 Detección y análisis de valores nulos

> Bloque con sangría



1.a. ¿Qué proporción de valores nulos presenta cada feature?

In [ ]:
# Borramos columnas extras
CleanInitialEducationByID = CleanInitialEducation.drop(["Country Name", "Country Code", "Indicator", "Features"], axis=1)
# Agrupamos por el codigo del indicador
CleanInitialEducationByID = CleanInitialEducationByID.groupby(["Indicator Code"]).count()
# Agregamos el pje de registros, sumando toda la info de todos los años y dividiendo por 29, dado que tenemos 29 años en consideración
CleanInitialEducationByID["Porcentaje de Registros"] = (CleanInitialEducationByID.sum(axis=1)/29)*100
# Ordenamos de mayor a menor pje de registros
CleanInitialEducationByID = CleanInitialEducationByID["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=False)

CleanInitialEducationByID

Disponemos de un muy buen porcentaje de registros por feature, considerando la totalidad de los años una vez realizado el filtrado hasta 1990.

1.b. ¿Los mismos se presentan aislados o agrupados por intervalos de tiempo?


En el dataframe CleanInitialEducation puede apreciarse como los pocos NaNs existentes se concentran hasta 1997 y en 2018.

AGREGAR GRAFICO DONDE SE VEAN QUIEBRES DE INFO

In [ ]:
# Aplicamos melt para tener los años en una unica columna y poder establecer intervalos para luego calcular el porcentaje de NaNs en cada uno de ellos.
CleanInitialEducationByYear = pd.melt(CleanInitialEducation.drop(["Country Code", "Country Name", "Indicator", "Indicator Code"], axis=1), id_vars = ["Features"])
CleanInitialEducationByYear["variable"] = pd.to_numeric(CleanInitialEducationByYear["variable"] )
CleanInitialEducationByYear = CleanInitialEducationByYear.rename(columns = {"variable": "Year"})

# El primer intervalo va de 1990 a 1997 inclusive
TotalData_1997 = (CleanInitialEducationByYear["Year"]<=1997).sum()
WithData_1997 = ((CleanInitialEducationByYear["Year"]<=1997) & (CleanInitialEducationByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1990 a 1997 es de: \n" +  str(round((1 - (WithData_1997/TotalData_1997))*100,2)) + "%") 

# El segundo intervalo va de 1998 a 2017 inclusive
TotalData_2017 = ((1998<=CleanInitialEducationByYear["Year"])&(CleanInitialEducationByYear["Year"]<=2017)).sum()
WithData_2017= (((1998<=CleanInitialEducationByYear["Year"])&(CleanInitialEducationByYear["Year"]<=2017)) & (CleanInitialEducationByYear["value"]>0)).sum()

print("El % de registros sin datos para el período de 1997 a 2017 es de: \n" +  str(round((1 - (WithData_2017/TotalData_2017))*100,2)) + "%") 

# El tercer intervalo unicamente comprende 2018
TotalData_2018 = (CleanInitialEducationByYear["Year"]>=2018).sum()
WithData_2018 = ((CleanInitialEducationByYear["Year"]>=2018) & (CleanInitialEducationByYear["value"]>0)).sum()

print("El % de registros sin datos para el año 2018 es de: \n" +  str(round((1 - (WithData_2018/TotalData_2018))*100,2)) + "%") 

1.c. ¿Cómo es la distribución de valores nulos por columna (es decir, para cada año)?

In [ ]:
# Calculamos proporción de NaNs por año en cada indicador
NotNanEducation=((CleanInitialEducation.drop(["Country Name", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Features").count()))
NotNanEducation['not_nan_values_count'] = NotNanEducation.apply(lambda x: x.sum(), axis=1)
plt.figure(figsize=(20, 10))
sns.heatmap(NotNanEducation.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en Argentina")

Del heatmap anterior se desprende que son los mismos features los que tienen valores faltantes en los distintos años. Es decir, por cada una de las columnas o para cada uno de los años, los valores nulos se encuentran mayoritariamente en los mismos features, salvo excepciones puntuales. Es interesante observar como, de los features con valores faltantes, todos tienen más de un valor faltante, y todos presentan valor faltante para los años 1992, 1993, 1995 y 2018. Partiendo de esta base, hay algunas particularidades de cada feature. Por su parte, el ultimo feature del heatmap, no presenta valores para un intervalo de tiempo no despreciable, desde 1990 a 1997.

1.d. ¿La presencia de ciertos valores nulos podría estar justificada por algún contexto social-político-económico a nivel nacional, regional o global?.

Tal como se observa en el mapa de calor anterior, desde 1992 se incrementa la cantidad de valores nulos para muchos de los features. Esto puede vincularse con las reformas educativas que sucedieron en esa década; particularmente las leyes de reforma de 1992, 1994 y 1995. Algunos de los cambios del sistema educativo se vinculan con la descentralización del control de las escuelas de Nación a provincias y la modificación de los años de cada nivel escolar (se creó la etapa de educación general básica de nueve años y el podimodal de tres años a continuación de ésta). Estos cambios pueden haber alterado la recolección de datos o, en menor medida y eventualmente, la forma de cálculo de los features.

Veamos si el resto de los países tiene datos de los features con NaNs de 1992 a 1995, de manera de corroborar la hipotesis del impacto de la reforma educativa.

In [ ]:
# Calculamos lista con features en los que Arg. no tiene valores. Teniendo en cuenta que el total de años es 29, de ahí se impone la condición.
NotNanEducationAux = NotNanEducation.reset_index()
FeaturesWithNanArg = NotNanEducationAux[NotNanEducationAux["not_nan_values_count"]<29]["Features"].to_list()

# Obtenemos DataFrame de los features en cuestión para el resto de los países
RelevantYearsAndFeaturesEducation = RelevantYearsEducation[RelevantYearsEducation["Features"].isin(FeaturesWithNanArg)]

# Heatmap similar al anterior para el resto de los países.
countries = RelevantYearsAndFeaturesEducation["Country Name"].drop_duplicates().to_list()
countries.remove("Argentina")

for countries in countries:
    NotNanEducationCountries=((RelevantYearsAndFeaturesEducation[RelevantYearsAndFeaturesEducation["Country Name"]==countries].drop(["Country Name", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Features").count()))
    NotNanEducationCountries['not_nan_values_count'] = NotNanEducationCountries.apply(lambda x: x.sum(), axis=1)
    plt.figure(figsize=(20, 10))
    sns.heatmap(NotNanEducationCountries.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
    plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en " + countries)

En los gráficos anteriores se observa cómo, dejando de lado Brasil (que practicamente no tiene datos para ningun feature previo a 1998), son pocos los países que no tienen valores (en los features donde no hay datos en Argentina) para 1992 y 1993 (sólo Ecuador y para unos pocos features), alejandose de la situación de Argentina, mientras que son más los países que presentan valores faltantes para 1994/1995 (Venezuela, Chile, Suriname).

Consecuentemente, podria decirse que una parte de los valores faltantes en Argentina si puede estar relacionada con la reforma (principalmente en 1992 y 1993), mientras que otra parte quizas puede relacionarse con alguna situación de la región dado que algunos países tampoco disponen valores (aunque no es posible advertir un comportamiento generalizado de falta de datos en todos los países; son bastante heterogeneos). En términos generales, en la década de los 90 hubo varias reformas educativas de la indole de la Argentina en el resto de los países latinoamericanos; puede que eso explique la falta de datos (variada para los años segun países) en este decenio.

Por su parte, la falta de valores nulos para el año 2018 no pareciera obedecer a un contexto particular, sino que más bien es un año reciente para el cual pueden aun no calcularse algunos de los features.

#### 2 Cálculo de estadísticos y distribuciones

2.a. Obtener la media, mediana, moda, desviación estándar, varianza, valor máximo y mínimo, percentiles y cualquier otro estadístico que les resulte importante de cada uno de los features de cada Indicador. 

In [ ]:
# Calculamos estadisticos por feature para la totalidad de los años
CleanInitialEducationByYear.drop("Year", axis=1).groupby(["Features"]).describe()

2.b. Realizar nuevamente los histogramas ó boxplots de las 15 features para cada Indicador.

In [ ]:
CleanInitialEducationByYear = CleanInitialEducationByYear.copy()

features = CleanInitialEducationByYear["Features"].drop_duplicates().to_list()

for features in features:
    plt.figure(figsize=(10,6))
    sns.boxplot(data=CleanInitialEducationByYear[CleanInitialEducationByYear["Features"]==features], x="Features", y="value", width=0.1)
    plt.title("Boxplot por feature", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.ylabel('Valor', fontsize=16)

2.c. Trazar los gráficos de línea de los 15 features en el intervalo considerado.

In [ ]:
CleanInitialEducationByYear = CleanInitialEducationByYear.copy()

features = CleanInitialEducationByYear["Features"].drop_duplicates().to_list()

for features in features:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=CleanInitialEducationByYear[CleanInitialEducationByYear["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=16)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Año', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

2.d. Analizar en conjunto los resultados de los puntos a, b y c. 

De los puntos a, b y c se desprenden las siguientes apreciaciones:

- Los features son sumamente heterogeneos en cuanto a sus valores dado que representan diferentes conceptos; existen porcentajes varios y cantidades.
- Cada feature es singular en cuanto al rango de sus datos, si bien algunos poseen más variabilidad en los datos por año que otros, son pocos los features que presentan outliers. Sólo 3 de los 15 features presentan valores atipicos.
- Las tendencias de los features son mayoritariamente distintas, sin poder generalizar un comportamiento para todos los features del indicador. Sin embargo, algunos features particulares, por grupos, presentan tendencias similares.

Un primer grupo, presenta identicas tendencias, sumamente cambiantes antes de 2010 donde ya se consolida una tendencia decreciente. En este grupo tenemos:

- Inscripción escolar, nivel primario, varones (% bruto)

- Inscripción escolar, nivel primario, mujeres (% bruto)

- Inscripción escolar, nivel primario (% bruto)
Un segundo grupo, presenta una caida en su tendencia a fines de los 90´s y permanece inalterada desde entonces. Aqui tenemos a la "Educación de nivel primario, duración (años)", que cae de 7 a 6 fruto de la reforma educativa, y la "Edad de ingreso a la escuela secundaria (años)", que cae de 13 a 12 por la misma causa.

Un tercer grupo, de un unico feature, presenta un aumento en su tendencia a fines de la decada del 90 y permanece inalterada desde ese momento. Aquí, tenemos a la "Educación de nivel secundario, duración (años)", que sube de 5 a 6, siendo el espejo del feature "Educación de nivel primario, duración (años)".

Un cuarto grupo, compuesto por un unico feature, no tiene cambios en su tendencia, se trata de la "Edad de inicio de la educación de nivel primario"; que siempre permaneció en 6 años.

Un quinto grupo, presenta tendencias crecientes en el ultimo tiempo. Aqui tenemos a los siguientes features:

- Población entre 15 y 64 años de edad (% del total)

- Educación de nivel secundario, alumnos en programas de educacion general

- Educación de nivel secundario, alumnos

- Inscripción escolar, nivel preprimario (% bruto)
Un sexto grupo, compuesto por un unico feature, presenta una tendencia decreciente en el ultimo tiempo, tratandose de la "Población entre 0 y 14 años de edad (% del total)". Se trata de otro espejo, esta vez del feature "Población entre 15 y 64 años de edad (% del total)" que crece en el tiempo.

Un septimo y ultimo grupo presenta una tendencia sumamente variable, sin poder establecer un patrón. Aqui se incluye a:

- Educacion de nivel primario, alumnos (% de mujeres)

- Educación de nivel primario, alumnos

- Proporción de inscripciones de mujeres con respecto a valores en la educación primaria (%)

#### 3 Análisis de posible imputación de valores faltantes

3.a. En base al estudio y análisis anterior, ¿Considera que se dispone de suficiente información para completar/rellenar los valores faltantes? ¿Cuáles reemplazaría y qué criterio/s usaría? Justifique sus respuestas.

La principal falta de datos se da entre 1992 a 1995 y en 2018. De esta manera, se deben tomar decisiones en torno a dos periodos de tiempo.

- Dado que en los años previos a 2018 todos los features tienen datos (desde 1997), entonces podrian rellenarse los datos para ese año. El error en el relleno será menor cuando la tendencia de los features no haya variado mucho en los años previos. Un criterio pertinente a utilizar es tomar la media de los últimos dos datos.
- La falta de datos de 1992 a 1995 requiere un analisis más detallado dado que no se disponen de suficientes datos antes de 1992 y se trata de 4 años en los cuales los features pueden haber variado por el impacto de la reforma educativa. Sin embargo y a simple vista, pueden observarse dos cosas: primero, los features con impacto directo de la reforma educativa si tienen datos (los referidos a los años de educacion de cada nivel) y, segundo, los gráficos de tendencia evidencian que la variación de los datos año tras año en aquellos features con NaNs no es sustancial de manera que no se incurriria en un gran error al interpolar los valores.

El objetivo será, entonces, completar los valores faltantes de manera de trabajar con el dataset desde 1990 a 2018

3.b. Identificar los casos en los cuáles no se cuenta con información suficiente para compensar la falta de valores nulos y analizar dichos features valiendose del análisis y visualización realizada en el práctico anterior:
- ¿Cómo es la distribución de este feature en los demás países? 
- ¿Presentan la misma tendencia a lo largo del tiempo? ¿Presentan la misma cantidad de valores faltantes? 
- ¿Es posible establecer alguna relación entre los países vecinos a Argentina o con algún país en particular?
- ¿De qué manera la posible correlación con otros países se encuentra limitada por el contexto local? Por ejemplo: La crisis del 2001 en Argentina es un evento totalmente local que probablemente no haya afectado a la región).

Dado que el objetivo propuesto es trabajar con los datos desde 1990 a 2018, el único feature que compromete el dataset es el llamado "Educación de nivel secundario, alumnos en programas de educación general", que no dispone de datos de 1990 hasta 1997 (Ni tampoco en 2018). En este caso, interpolando hacia atrás de 1997 podriamos incurrir en un error grande dado que son muchos años. Lo óptimo sería eliminar este feature de manera de poder trabajar con la totalidad de los años para el resto de los features, desde 1990 a 2018.

Veamos cual es el comportamiento del feature en la región.

In [ ]:
ComplicatedFeatureEducation = RelevantYearsEducation[RelevantYearsEducation["Features"]=="Educación de nivel secundario, alumnos en programas de educación general"]
NotNanComplicatedFeatureEducation=((ComplicatedFeatureEducation.drop(["Features", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Country Name").count()))
NotNanComplicatedFeatureEducation['not_nan_values_count'] = NotNanComplicatedFeatureEducation.apply(lambda x: x.sum(), axis=1)
plt.figure(figsize=(20, 10))
sns.heatmap(NotNanComplicatedFeatureEducation.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
plt.title("Mapa de Calor: Proporción de Valores NaN del Feature <Educación de nivel secundario, alumnos en programas de educación general>")

Como se observa, el feature presenta valores faltantes para la mayoria de los países antes de 1997. Este comportamiento, sin embargo, no es algo aislado sino que es comun al comportamiento del indicador, que se adelantó en el práctico anterior posee la mayor cantidad de NaNs hasta 1997 Se visualiza que Argentina es el único país que no presenta datos desde 1990 hasta 1997, siendo uno de los países más complicados en la década de 1990 seguido por Bolivia.

In [ ]:
ComplicatedFeatureEducationAux = pd.melt(ComplicatedFeatureEducation.drop(["Country Code", "Indicator", "Indicator Code", "Features"], axis=1), id_vars = ["Country Name"])
ComplicatedFeatureEducationAux["variable"] = pd.to_numeric(ComplicatedFeatureEducationAux["variable"] )
ComplicatedFeatureEducationAux = ComplicatedFeatureEducationAux.rename(columns = {"variable": "Year"})

countries = ComplicatedFeatureEducationAux["Country Name"].drop_duplicates().to_list()

for countries in countries:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=ComplicatedFeatureEducationAux[ComplicatedFeatureEducationAux["Country Name"]==countries], x="Year", y="value")
    plt.title("Serie Temporal del Feature Complicado\n" + str(countries), fontsize=16)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Año', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

En relación a las tendencias, por su parte, para muchos países pareciera que el feature crece en el tiempo. Sin embargo, no es posible establecer un patrón general de los países, de manera que sería dificil intentar extrapolar los datos de otro país a Argentina en este contexto. Por lo tanto se prosigue con la decisión de desechar este feature problematico.

3.c y 3.d. Conclusiones de analisis y limpieza final

Enumerar las conclusiones que se extrajeron a partir de los puntos a y b y tomar una decisión al respecto, es decir:
- Imputar valores nulos usando el criterio elegido.
- Eliminar aquellos features con valores nulos por carecer de información suficiente para tomar una decisión.

Dejar un registro en formato resumen, o el que consideren pertinente, de las decisiones tomadas junto con alguna breve explicación de las mismas. La idea es que sea lo más entendible posible para un lector técnico pero sin demasiado conocimiento del dataset.

Como se adelantó, se completarán los valores faltantes para los features que corresponda de manera de trabajar con los años de 1990 a 1970. En este proceso, se decide eliminar al feature "Educación de nivel secundario, alumnos en programas de educación general" dado que no dispone de datos en un extenso intervalo de tiempo.

En primer lugar eliminamos el feature mencionado:

In [ ]:
CleanEducationStepOne = CleanInitialEducation[CleanInitialEducation["Features"]!="Educación de nivel secundario, alumnos en programas de educación general"]
CleanEducationStepOne

A continuación se prosigue completando los NaNs

**AÑO 2018**: 

Para los features con valores faltantes, se decide completar el año 2018 bajo los siguientes criterios:
- Si no hay una tendencia clara en los últimos tres años previos a 2018 se toma un promedio ponderado de los tres años anteriores. Al dato de 2015 se lo pondera con un peso de 0.20, al de 2016 con un peso de 0.3 y al de 2017 con un peso de 0.5.
- Si hay una tendencia creciente o decreciente en los últimos tres años previos a 2018, se calcula la tasa de crecimiento o decrecimiento, respectivamente, y se la aplica sobre el dato de 2017 para obtener el dato de 2018.

In [ ]:
# Seleccionamos features sin Nans en el 2018 para no estimar tal año para ellos
CleanEducationStepOneNotNull = CleanEducationStepOne[CleanEducationStepOne["2018"].notnull()]
CleanEducationStepOneNotNull

# Seleccionamos features con NaNs en el 2018 y renombramos columnas para poder seleccionarlas más rapidamente
CleanEducationStepOneNull = CleanEducationStepOne[CleanEducationStepOne["2018"].isna()]
CleanEducationStepOneNull = CleanEducationStepOneNull.rename(columns={"2014":"T_Four", "2015":"T_Three", "2016":"T_Two", "2017":"T_One"})

# En caso que exista una tendencia creciente O decreciente, aplicamos la tasa de crecimiento promedio de los ultimos tres años para interpolar el dato faltante.
IncreasingTrend = ((CleanEducationStepOneNull.T_One - CleanEducationStepOneNull.T_Two) > 0) & ((CleanEducationStepOneNull.T_Two - CleanEducationStepOneNull.T_Three)>0) & ((CleanEducationStepOneNull.T_Three - CleanEducationStepOneNull.T_Four)>0)
DecreasingTrend = ((CleanEducationStepOneNull.T_One - CleanEducationStepOneNull.T_Two) < 0) & ((CleanEducationStepOneNull.T_Two - CleanEducationStepOneNull.T_Three)<0) & ((CleanEducationStepOneNull.T_Three - CleanEducationStepOneNull.T_Four)<0)

if ((IncreasingTrend)|(DecreasingTrend)) is True:
    GrowthRateEducationThreePreviousPeriods = (CleanEducationStepOneNull.T_Three - CleanEducationStepOneNull.T_Four)/(CleanEducationStepOneNull.T_Four)
    GrowthRateEducationTwoPreviousPeriods = (CleanEducationStepOneNull.T_Two - CleanEducationStepOneNull.T_Three)/(CleanEducationStepOneNull.T_Three)
    GrowthRateEducationOnePreviousPeriods = (CleanEducationStepOneNull.T_One - CleanEducationStepOneNull.T_Two)/(CleanEducationStepOneNull.T_Two)
    GrowthRateEducation = (GrowthRateEducationThreePreviousPeriods+GrowthRateEducationTwoPreviousPeriods+GrowthRateEducationOnePreviousPeriods)/3
    CleanEducationStepOneNull["2018"] = CleanEducationStepOneNull.T_One*(1+GrowthRateEducation)

# Caso contrario, si no hay una tendencia clara antes de 2018, aplicamos un promedio ponderado de los ultimos tres años. Los datos más recientes tienen mas peso.
else:
    CleanEducationStepOneNull["2018"] = CleanEducationStepOneNull.T_Three*0.2 + CleanEducationStepOneNull.T_Two*0.3 + CleanEducationStepOneNull.T_One*0.5
    
# Unimos dataframe sin Nans, con datos originales para el 2018, con el dataframe sobre el que se realizó la estimación del valor de 2018.
CleanEducationStepTwo = pd.concat([CleanEducationStepOneNotNull, CleanEducationStepOneNull.rename(columns={"T_Three":"2015", "T_Two":"2016", "T_One":"2017", "T_Four":"2014"})], ignore_index=False)
CleanEducationStepTwo


**AÑOS 1992 A 1997:**

Se extrapola con función interpolate de pandas. Primero se analiza qué tipo de metodo de interpolación es más acorde.

In [ ]:
# Aplicamos melt y renombramos columnas para realizar la interpolación
CleanEducationStepTwoByYear = pd.melt(CleanEducationStepTwo, id_vars = ["Features", "Indicator", "Country Name", "Country Code", "Indicator Code"])
CleanEducationStepTwoByYear["variable"] = pd.to_numeric(CleanEducationStepTwoByYear["variable"] )
CleanEducationStepTwoByYear = CleanEducationStepTwoByYear.rename(columns = {"variable": "Year"})
# Transformamos el formato de los años a datatime
CleanEducationStepTwoByYear['Year'] = pd.to_datetime(CleanEducationStepTwoByYear['Year'], format='%Y').dt.year
# Ordenamos por feature y año para interpolar correctamente y no mezclar valores de features
CleanEducationStepThreeByYearAux = CleanEducationStepTwoByYear.sort_values(by=["Features", "Year"])

features = CleanEducationStepThreeByYearAux["Features"].drop_duplicates().to_list()
years = CleanEducationStepThreeByYearAux["Year"].drop_duplicates().to_list()

# Interpolamos con distintos métodos para analizar qué método se adapta mejor a cada feature
for features in features:
    methods = ["linear", "nearest", "quadratic", "cubic", "akima"]
    df = pd.DataFrame({m: CleanEducationStepThreeByYearAux[CleanEducationStepThreeByYearAux["Features"]==features]["value"].interpolate(method=m) for m in methods})
    df.plot()
    plt.title(features)
    plt.legend(loc='upper center', bbox_to_anchor=(1.2, 1), shadow=True)
    plt.xticks([])


Como se observa en los gráficos anteriores, los métodos de interpolación cuadrática y cúbica tienden a generar valores alejados al resto de los métodos para los años que nos importan (1992 a 1995, con las particularidades para cada feature con distintos NaNs en este intervalo). Se trata de interpolaciones más arriesgadas, en las cuales, tal como evidencian las figuras, la amplitud de los picos posteriores (en el tiempo) se repiten en el pasado (en este caso en donde tenemos valores faltantes). Si bien puede que las series temporales se comporten de esta forma reiterativa, en este caso optamos por descartar estas interpolaciones para ser más conservadores con nuestro analisis.

Los métodos de interpolación lineal y akima arrojan resultados similares, siendo que el segundo método suaviza un poco más el comportamiento de la serie. Si bien el método akima podría ser una buena aproximación, seleccionaremos el método del valor más cercano presuponiendo que el valor faltante en cierto año es igual al valor del año anterior. Es un criterio conservador que, como se observa en las series, da en muchos casos una tendencia de escalera.

In [ ]:
CleanEducationStepThree = CleanEducationStepTwoByYear.sort_values(by=["Features", "Year"]).interpolate(method = "nearest")
CleanEducationStepThree 

In [ ]:
CleanEducationStepThree.isna().sum()

3.e. Dataset Final

A partir de los resultados del punto c, generar un nuevo dataset por Indicador para Argentina con un nombre adecuado y guardarlo en formato csv. El procedimiento puede realizarse con la función to_csv de pandas.

In [ ]:
CleanEducationStepThree.to_csv("Clean Education")



```
# This is formatted as code
```

## PROTECCIÓN SOCIAL

### Parte 1: Limpieza

1. Eliminar todos los datos previos al año 1990 (es decir, la franja temporal desde 1970 hasta 1989).

In [ ]:
columns = social_protection.columns
columnsDelete = columns[5:25].to_list()
columnsDelete = columnsDelete + [columns[-1]]

In [ ]:
RelevantYearsSocialProtection = social_protection.drop(columnsDelete, axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)

RelevantYearsSocialProtection

2. Filtrar los datos anteriores considerando solo Argentina.

In [ ]:
RelevantYearsSocialProtectionArg = RelevantYearsSocialProtection[RelevantYearsSocialProtection["Country Name"]=="Argentina"]
RelevantYearsSocialProtectionArg

3. Restringir el conjunto de las variables del indicador a las 15 features elegidas (aquellas con más del 50% de valores no nulos) durante el desarrollo del Práctico anterior.

Primero se busca de nuevo la lista con los 15 features:

In [ ]:
SocialProtectionValuesByICode = social_protection.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "Features"], axis=1)
SocialProtectionValuesByICode["Valor"] = SocialProtectionValuesByICode.sum(axis=1)
SocialProtectionValuesByICode = SocialProtectionValuesByICode.sort_values(by=['Valor'], ascending=False)
SocialProtectionValuesByICode = ((SocialProtectionValuesByICode/12)*100).drop(columns = ["Valor"])
SocialProtectionValuesByICode = SocialProtectionValuesByICode.rename(columns = lambda x : str(x)[:4])
round(SocialProtectionValuesByICode,2)

In [ ]:
SocialProtectionValuesByICode=SocialProtectionValuesByICode.reset_index()
SocialProtectionValuesByICode["Features"] = SocialProtectionValuesByICode["Indicator Code"]
SocialProtectionValuesByICode["Features"] = SocialProtectionValuesByICode["Features"].map(education.set_index("Indicator Code")["Features"].to_dict())
FeaturesSocialProtectionSelection = SocialProtectionValuesByICode.iloc[0:15][["Indicator Code", "Features"]].set_index("Indicator Code")
FeaturesSocialProtectionSelection

In [ ]:
SocialProtectionIDCode = FeaturesSocialProtectionSelection.reset_index()['Indicator Code'].to_list()

CleanInitialSocialProtection = RelevantYearsSocialProtectionArg[RelevantYearsSocialProtectionArg["Indicator Code"].isin(SocialProtectionIDCode)]
CleanInitialSocialProtection

4. Realizar una inspección acerca del significado de las variables valiendose de la información disponible en la metadata. Pueden guiarse de las siguientes preguntas:

- ¿Son todos los features representativos dentro del contexto argentino? ¿Existe alguno/s que podría no considerarse dado que no aplica o no aplicó a la realidad argentina? Justifique las respuestas.
- En las variables que involucran género y edad: ¿se consideran todos casos?. Por ejemplo: % de trabajadores asalariados hombres y % de trabajadoras asalariaradas mujeres. Si la respuesta es negativa, ¿cuál consideran que sería la decisión más acertada para abordar este aspecto?

In [ ]:
MetadataEducationRelevantFeaturesSP = metadata_socialprotection[metadata_socialprotection["INDICATOR_CODE"].isin(SocialProtectionIDCode)].drop("Unnamed: 0", axis=1)
MetadataEducationRelevantFeaturesSP

Agregar analisis del punto anterior!

### Parte 2: Análisis y Curación

#### 1 Detección y análisis de valores nulos

1.a. ¿Qué proporción de valores nulos presenta cada feature?

In [ ]:
# Borramos columnas extras
CleanInitialSocialProtectionByID = CleanInitialSocialProtection.drop(["Country Name", "Country Code", "Indicator", "Features"], axis=1)
# Agrupamos por el codigo del indicador
CleanInitialSocialProtectionByID = CleanInitialSocialProtectionByID.groupby(["Indicator Code"]).count()
# Agregamos el pje de registros, sumando toda la info de todos los años y dividiendo por 29, dado que tenemos 29 años en consideración
CleanInitialSocialProtectionByID["Porcentaje de Registros"] = (CleanInitialSocialProtectionByID.sum(axis=1)/29)*100
# Ordenamos de mayor a menor pje de registros
CleanInitialSocialProtectionByID = CleanInitialSocialProtectionByID["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=False)

CleanInitialSocialProtectionByID

Disponemos de un muy buen porcentaje de registros por feature, considerando la totalidad de los años una vez realizado el filtrado hasta 1990.

1.b. ¿Los mismos se presentan aislados o agrupados por intervalos de tiempo?


En el dataframe CleanInitialEducation puede apreciarse como los pocos NaNs existentes se concentran hasta 1997 y en 2018.

AGREGAR GRAFICO DONDE SE VEAN QUIEBRES DE INFO

In [ ]:
# Aplicamos melt para tener los años en una unica columna y poder establecer intervalos para luego calcular el porcentaje de NaNs en cada uno de ellos.
CleanInitialSocialProtectionByYear = pd.melt(CleanInitialSocialProtection.drop(["Country Code", "Country Name", "Indicator", "Indicator Code"], axis=1), id_vars = ["Features"])
CleanInitialSocialProtectionByYear["variable"] = pd.to_numeric(CleanInitialSocialProtectionByYear["variable"] )
CleanInitialSocialProtectionByYear = CleanInitialSocialProtectionByYear.rename(columns = {"variable": "Year"})

# El primer intervalo va de 1990 a 1997 inclusive
TotalData_1997 = (CleanInitialSocialProtectionByYear["Year"]<=1997).sum()
WithData_1997 = ((CleanInitialSocialProtectionByYear["Year"]<=1997) & (CleanInitialSocialProtectionByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1990 a 1997 es de: \n" +  str(round((1 - (WithData_1997/TotalData_1997))*100,2)) + "%") 

# El segundo intervalo va de 1998 a 2017 inclusive
TotalData_2017 = ((1998<=CleanInitialSocialProtectionByYear["Year"])&(CleanInitialSocialProtectionByYear["Year"]<=2017)).sum()
WithData_2017= (((1998<=CleanInitialSocialProtectionByYear["Year"])&(CleanInitialSocialProtectionByYear["Year"]<=2017)) & (CleanInitialSocialProtectionByYear["value"]>0)).sum()

print("El % de registros sin datos para el período de 1997 a 2017 es de: \n" +  str(round((1 - (WithData_2017/TotalData_2017))*100,2)) + "%") 

# El tercer intervalo unicamente comprende 2018
TotalData_2018 = (CleanInitialSocialProtectionByYear["Year"]>=2018).sum()
WithData_2018 = ((CleanInitialSocialProtectionByYear["Year"]>=2018) & (CleanInitialSocialProtectionByYear["value"]>0)).sum()

print("El % de registros sin datos para el año 2018 es de: \n" +  str(round((1 - (WithData_2018/TotalData_2018))*100,2)) + "%") 

1.c. ¿Cómo es la distribución de valores nulos por columna (es decir, para cada año)?

In [ ]:
# Calculamos proporción de NaNs por año en cada indicador
NotNanSocialProtection=((CleanInitialSocialProtection.drop(["Country Name", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Features").count()))
NotNanSocialProtection['not_nan_values_count'] = NotNanSocialProtection.apply(lambda x: x.sum(), axis=1)
plt.figure(figsize=(20, 10))
sns.heatmap(NotNanSocialProtection.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en Argentina")

Como se observa en el heatmap, la variable desempleo (expresado en porcentaje de participacion de acuerdo al genero en la fuerza laboral) tanto para varones como para mujeres son las variables que presentan datos faltantes en el año 2007, 2015 y 2016. No pareceria ser muy grave ya que para los años que se encuentran cercanos a los nombrados si se dispone de datos. Desmpleo total es la otra variable que no presenta datos para los años 2015 y 2016.

1.d. ¿La presencia de ciertos valores nulos podría estar justificada por algún contexto social-político-económico a nivel nacional, regional o global?.

Los datos faltantes del año 2007 podrian estar asociados a la intervencion del Indec en Argentina. En el caso de los años 2015 y 2016 probablemente respondan mas a un cambio de gobierno (quizas cambio en la metodologia de medicion?). De todas maneras parece ser un problema o una situacion particular de Argentina. Para comprobarlo vamos a ver como se comportaron estas variables en el resto de los paises del Mercosur para el período analizado.

In [ ]:
# Calculamos lista con features en los que Arg. no tiene valores. Teniendo en cuenta que el total de años es 29, de ahí se impone la condición.
NotNanSocialProtectionAux = NotNanSocialProtection.reset_index()
FeaturesWithNanArgSP = NotNanSocialProtectionAux[NotNanSocialProtectionAux["not_nan_values_count"]<29]["Features"].to_list()

# Obtenemos DataFrame de los features en cuestión para el resto de los países
RelevantYearsAndFeaturesSP = RelevantYearsSocialProtection[RelevantYearsSocialProtection["Features"].isin(FeaturesWithNanArgSP)]

# Heatmap similar al anterior para el resto de los países.
countries = RelevantYearsAndFeaturesSP["Country Name"].drop_duplicates().to_list()
countries.remove("Argentina")

for countries in countries:
    NotNanSocialProtectionCountries=((RelevantYearsAndFeaturesSP[RelevantYearsAndFeaturesSP["Country Name"]==countries].drop(["Country Name", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Features").count()))
    NotNanSocialProtectionCountries['not_nan_values_count'] = NotNanSocialProtectionCountries.apply(lambda x: x.sum(), axis=1)
    plt.figure(figsize=(20, 10))
    sns.heatmap(NotNanSocialProtectionCountries.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
    plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en " + countries)

Como puede observarse en los diferentes heatmaps no pareceria haber ninguna tendencia en la falta de informacion para las variables analizadas. Hay paises donde hay muchos años con datos faltantes, caso de Guyana o Suriname, y paises como Chile donde casi no hay datos faltantes. Por lo tanto no se puede colcuir en la existencia de algun patron particular de datos fatantes.

#### 2 Cálculo de estadísticos y distribuciones

2.a. Obtener la media, mediana, moda, desviación estándar, varianza, valor máximo y mínimo, percentiles y cualquier otro estadístico que les resulte importante de cada uno de los features de cada Indicador. 

In [ ]:
# Calculamos estadisticos por feature para la totalidad de los años
CleanInitialSocialProtectionByYear.drop("Year", axis=1).groupby(["Features"]).describe()

2.b. Realizar nuevamente los histogramas ó boxplots de las 15 features para cada Indicador.

In [ ]:
CleanInitialSocialProtectionByYear = CleanInitialSocialProtectionByYear.copy()

features = CleanInitialSocialProtectionByYear["Features"].drop_duplicates().to_list()

for features in features:
    plt.figure(figsize=(10,6))
    sns.boxplot(data=CleanInitialSocialProtectionByYear[CleanInitialSocialProtectionByYear["Features"]==features], x="Features", y="value", width=0.1)
    plt.title("Boxplot por feature", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.ylabel('Valor', fontsize=16)

2.c. Trazar los gráficos de línea de los 15 features en el intervalo considerado.

In [ ]:
CleanInitialSocialProtectionByYear = CleanInitialSocialProtectionByYear.copy()

features = CleanInitialSocialProtectionByYear["Features"].drop_duplicates().to_list()

for features in features:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=CleanInitialSocialProtectionByYear[CleanInitialSocialProtectionByYear["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=16)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Año', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

2.d. Analizar en conjunto los resultados de los puntos a, b y c. 

Analisis de los puntos a, b y c se:

Como se pueden ver en los diferentes gráficos, las variables son totalmente heterogeneas en su comportamiento, sin poder identificarse un patron de comportamiento.

Observando los boxplot en general la mayoria presenta una gran dispersion de datos determinado por el tamaño de la caja y los percentiles 25 y 75 en los limites de la caja y por los min y maximos en las longitu de los whisker. Esto habla de la dispersion que tienen los datos en su distribucion.

Pocas variables, como el caso de de la tasa de poblacion activa de varones, presentan valores extremos u outliers.

Algunas variables como las distintas tassas de participacion en las distintas fuerzas laborales parecieran tener una marcada tendencia decreciente conforme trascurren los años.


#### 3 Análisis de posible imputación de valores faltantes

3.a. En base al estudio y análisis anterior, ¿Considera que se dispone de suficiente información para completar/rellenar los valores faltantes? ¿Cuáles reemplazaría y qué criterio/s usaría? Justifique sus respuestas.

La principal falta de datos se da en el año 2007 y entre los años 2015 y 2016.

Debido a que se cuenta con infromacion de los años que rodean a aquellos (2014 y 2017 en los casos mas complicados) con datos faltantes, se procederá a rellenar los datos faltantes.

3.b. Identificar los casos en los cuáles no se cuenta con información suficiente para compensar la falta de valores nulos y analizar dichos features valiendose del análisis y visualización realizada en el práctico anterior:
- ¿Cómo es la distribución de este feature en los demás países? 
- ¿Presentan la misma tendencia a lo largo del tiempo? ¿Presentan la misma cantidad de valores faltantes? 
- ¿Es posible establecer alguna relación entre los países vecinos a Argentina o con algún país en particular?
- ¿De qué manera la posible correlación con otros países se encuentra limitada por el contexto local? Por ejemplo: La crisis del 2001 en Argentina es un evento totalmente local que probablemente no haya afectado a la región).

Como se mostro en el heatmap mas arriba la falta de información de estas variables pareceria ser mas una situacion local de Argentina, quizas relacionada a lo politico. En la región no se observa un patron de comportamiento para los mismas variables teniendo paises con informacion todos los años y paises donde faltan datos en la mayoria del periodo analizado.

A continuación un heatmap para la region:

In [ ]:
ComplicatedFeatureSocialProtection = RelevantYearsSocialProtection[RelevantYearsSocialProtection["Features"]=="Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)"]
NotNanComplicatedFeatureSocialProtection=((ComplicatedFeatureSocialProtection.drop(["Features", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Country Name").count()))
NotNanComplicatedFeatureSocialProtection['not_nan_values_count'] = NotNanComplicatedFeatureSocialProtection.apply(lambda x: x.sum(), axis=1)
plt.figure(figsize=(20, 10))
sns.heatmap(NotNanComplicatedFeatureSocialProtection.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
plt.title("Mapa de Calor: <Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)>")

En general no hay un patron o paises que se comporten como Argentina para los años 2015 y 2016. La excepcion serian los paises mencionados anteriormente en donde la falta de datos es mas contundente y no de esos años particulares (caso de Guayana o Venezuela).

In [ ]:
ComplicatedFeatureSocialProtectionAux = pd.melt(ComplicatedFeatureSocialProtection.drop(["Country Code", "Indicator", "Indicator Code", "Features"], axis=1), id_vars = ["Country Name"])
ComplicatedFeatureSocialProtectionAux["variable"] = pd.to_numeric(ComplicatedFeatureSocialProtectionAux["variable"] )
ComplicatedFeatureSocialProtectionAux = ComplicatedFeatureSocialProtectionAux.rename(columns = {"variable": "Year"})

countries = ComplicatedFeatureSocialProtectionAux["Country Name"].drop_duplicates().to_list()

for countries in countries:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=ComplicatedFeatureSocialProtectionAux[ComplicatedFeatureSocialProtectionAux["Country Name"]==countries], x="Year", y="value")
    plt.title("Serie Temporal del Feature Complicado\n" + str(countries), fontsize=16)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Año', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

Si bien pareceria haber una tendendia decreciente a lo largo de los años para los distintos paises, sin embargo, el comportamiento heterogeneo y la falta de datos en algunos paises hace un tanto dificil poder asegurar dicha conclusión.

3.c y 3.d. Conclusiones de analisis y limpieza final

Se completarán los valores faltantes para los features que corresponda de manera de trabajar con los años de 1990 a 2018.

Primero completamos 2015 y 2016 usando la media ponderada de los ultimos tres años, con distinto peso cada uno de manera de dar mayor importancia al dato mas reciente.

**AÑO 2015 y 2016**: 



In [ ]:
#### PRIMER CRITERIO DE IMPUTACIÓN DE DATOS
# Seleccionamos features con NaNs
SPFeatures_dataset_null = RelevantYearsAndFeaturesSP[RelevantYearsAndFeaturesSP["2015"].isnull()]
# Completamos los valores nulos de 2015 con los de 2014.
SPFeatures_dataset_null["2015"] = SPFeatures_dataset_null["2015"].fillna(SPFeatures_dataset_null["2014"])
# Redefinimos el dataset
SPFeatures_DATASET_step_one = SPFeatures_dataset_null


# Chequeamos que no existan features con nans para 2015
SPFeatures_DATASET_step_one["2015"].isnull().sum()

In [ ]:

SPFeatures_DATASET_step_two = pd.melt(SPFeatures_DATASET_step_one, id_vars = ["Features"])
SPFeatures_DATASET_step_two["variable"] = pd.to_numeric(SPFeatures_DATASET_step_two["variable"] )
SPFeatures_DATASET_step_two = SPFeatures_DATASET_step_two.rename(columns = {"variable": "Year"})
SPFeatures_DATASET_step_two['Year'] = pd.to_datetime(SPFeatures_DATASET_step_two['Year'], format='%Y').dt.year
SPFeatures_DATASET_step_two = SPFeatures_DATASET_step_two.sort_values(by =["Features", "Year"]).interpolate()


# Check de que todos los años tienen datos
SPFeatures_DATASET_step_two.isnull().sum()


3.e. Dataset Final

A partir de los resultados del punto c, generar un nuevo dataset por Indicador para Argentina con un nombre adecuado y guardarlo en formato csv. El procedimiento puede realizarse con la función to_csv de pandas.

## AMBIENTE

### Parte 1: Limpieza

1. Eliminar todos los datos previos al año 1990 (es decir, la franja temporal desde 1970 hasta 1989).

In [ ]:
columns = environment.columns
columnsDelete = columns[5:25].to_list()
columnsDelete = columnsDelete + [columns[-1]]

In [ ]:
RelevantYearsEnvironment = environment.drop(columnsDelete, axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)

A continuación se proponen una serie de consignas a llevar a cabo **sobre cada Indicador** con el objetivo de realizar la limpieza de los datos.

In [ ]:
RelevantYearsEnvironment

2. Filtrar los datos anteriores considerando solo Argentina.

In [ ]:
RelevantYearsEnvironmentArg = RelevantYearsEnvironment[RelevantYearsEnvironment["Country Name"]=="Argentina"]
RelevantYearsEnvironmentArg 

3. Restringir el conjunto de las variables del indicador a las 15 features elegidas (aquellas con más del 50% de valores no nulos) durante el desarrollo del Práctico anterior.

Primero se busca de nuevo la lista con los 15 features:

In [ ]:
EnvironmentValuesByICode = environment.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "Features"], axis=1)
EnvironmentValuesByICode["Valor"] = EnvironmentValuesByICode.sum(axis=1)
EnvironmentValuesByICode = EnvironmentValuesByICode.sort_values(by=['Valor'], ascending=False)
EnvironmentValuesByICode = ((EnvironmentValuesByICode/12)*100).drop(columns = ["Valor"])
EnvironmentValuesByICode = EnvironmentValuesByICode.rename(columns = lambda x : str(x)[:4])
round(EnvironmentValuesByICode,2)

In [ ]:
EnvironmentValuesByICode=EnvironmentValuesByICode.reset_index()
EnvironmentValuesByICode["Features"] = EnvironmentValuesByICode["Indicator Code"]
EnvironmentValuesByICode["Features"] = EnvironmentValuesByICode["Features"].map(environment.set_index("Indicator Code")["Features"].to_dict())
FeaturesEnvironmentSelection = EnvironmentValuesByICode.iloc[0:15][["Indicator Code", "Features"]].set_index("Indicator Code")
FeaturesEnvironmentSelection

In [ ]:
EnvironmentIDCode = FeaturesEnvironmentSelection.reset_index()['Indicator Code'].to_list()

CleanInitialEnvironment = RelevantYearsEnvironmentArg[RelevantYearsEnvironmentArg["Indicator Code"].isin(EnvironmentIDCode)]
CleanInitialEnvironment

4. Realizar una inspección acerca del significado de las variables valiendose de la información disponible en la metadata. Pueden guiarse de las siguientes preguntas:

- ¿Son todos los features representativos dentro del contexto argentino? ¿Existe alguno/s que podría no considerarse dado que no aplica o no aplicó a la realidad argentina? Justifique las respuestas.
- En las variables que involucran género y edad: ¿se consideran todos casos?. Por ejemplo: % de trabajadores asalariados hombres y % de trabajadoras asalariaradas mujeres. Si la respuesta es negativa, ¿cuál consideran que sería la decisión más acertada para abordar este aspecto?

In [ ]:
MetadataEnvironmentRelevantFeatures = metadata_environment[metadata_environment["INDICATOR_CODE"].isin(EnvironmentIDCode)].drop("Unnamed: 0", axis=1)
MetadataEnvironmentRelevantFeatures

Puede apreciarse que los features son representativos dentro del contexto argentino. No habria que eliminar ningun feature en este sentido.

### Parte 2: Análisis y Curación

#### 1 Detección y análisis de valores nulos

1.a. ¿Qué proporción de valores nulos presenta cada feature?

In [ ]:
# Borramos columnas extras
CleanInitialEnvironmentByID = CleanInitialEnvironment.drop(["Country Name", "Country Code", "Indicator", "Features"], axis=1)
# Agrupamos por el codigo del indicador
CleanInitialEnvironmentByID = CleanInitialEnvironmentByID.groupby(["Indicator Code"]).count()
# Agregamos el pje de registros, sumando toda la info de todos los años y dividiendo por 29, dado que tenemos 29 años en consideración
CleanInitialEnvironmentByID["Porcentaje de Registros"] = (CleanInitialEnvironmentByID.sum(axis=1)/29)*100
# Ordenamos de mayor a menor pje de registros
CleanInitialEnvironmentByID = CleanInitialEnvironmentByID["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=False)

CleanInitialEnvironmentByID

Disponemos de un muy buen porcentaje de registros por feature, considerando la totalidad de los años una vez realizado el filtrado hasta 1990.

1.b. ¿Los mismos se presentan aislados o agrupados por intervalos de tiempo?


En el dataframe CleanInitialEnvironment puede apreciarse como los NaNs se concentran en 2017 y 2018.

In [ ]:
# Aplicamos melt para tener los años en una unica columna y poder establecer intervalos para luego calcular el porcentaje de NaNs en cada uno de ellos.
CleanInitialEnvironmentByYear = pd.melt(CleanInitialEnvironment.drop(["Country Code", "Country Name", "Indicator", "Indicator Code"], axis=1), id_vars = ["Features"])
CleanInitialEnvironmentByYear["variable"] = pd.to_numeric(CleanInitialEnvironmentByYear["variable"] )
CleanInitialEnvironmentByYear = CleanInitialEnvironmentByYear.rename(columns = {"variable": "Year"})

# El primer intervalo va de 1990 a 2016 inclusive
TotalData_2016 = (CleanInitialEnvironmentByYear["Year"]<=2016).sum()
WithData_2016 = ((CleanInitialEnvironmentByYear["Year"]<=2016) & (CleanInitialEnvironmentByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1990 a 2016 es de: \n" +  str(round((1 - (WithData_2016/TotalData_2016))*100,2)) + "%") 

# El segundo intervalo va de 2017 a 2018 inclusive
TotalData_2018 = (CleanInitialEnvironmentByYear["Year"]>=2018).sum()
WithData_2018 = ((CleanInitialEnvironmentByYear["Year"]>=2018) & (CleanInitialEnvironmentByYear["value"]>0)).sum()

print("El % de registros sin datos para el año 2017 y 2018 es de: \n" +  str(round((1 - (WithData_2018/TotalData_2018))*100,2)) + "%") 

1.c.¿Cómo es la distribución de valores nulos por columna (es decir, para cada año)?

In [ ]:
# Calculamos proporción de NaNs por año en cada indicador
NotNanEnvironment=((CleanInitialEnvironment.drop(["Country Name", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Features").count()))
NotNanEnvironment['not_nan_values_count'] = NotNanEnvironment.apply(lambda x: x.sum(), axis=1)
plt.figure(figsize=(20, 10))
sns.heatmap(NotNanEnvironment.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en Argentina")

Del heatmap anterior se desprende que muchos de los features no poseen datos para 2018, mientras que un subgrupo de éstos tampoco presenta valores para 2017. Sin embargo, desde 1990 a 2016 no existe ningún NaN para ningun feature.

1.d. ¿La presencia de ciertos valores nulos podría estar justificada por algún contexto social-político-económico a nivel nacional, regional o global?.

No se trata de un fenomeno nacional particular dado que todos los países de la región tienen un comportamiento similar de datos faltantes para 2018 mayoritariamente y para 2017 en menor medida. No pareciera deberse a un contexto regional en relación al ambiente, sino más bien a una falta de datos debido a que son años muy recientes para los cuales aun no existen estimaciones.

In [ ]:
# Calculamos lista con features en los que Arg. no tiene valores. Teniendo en cuenta que el total de años es 29, de ahí se impone la condición.
NotNanEnvironmentAux = NotNanEnvironment.reset_index()
FeaturesEnvironmentWithNanArg = NotNanEnvironmentAux[NotNanEnvironmentAux["not_nan_values_count"]<29]["Features"].to_list()

# Obtenemos DataFrame de los features en cuestión para el resto de los países
RelevantYearsAndFeaturesEnvironment = RelevantYearsEnvironment[RelevantYearsEnvironment["Features"].isin(FeaturesEnvironmentWithNanArg)]

# Heatmap similar al anterior para el resto de los países.
countries = RelevantYearsAndFeaturesEnvironment["Country Name"].drop_duplicates().to_list()
countries.remove("Argentina")

for countries in countries:
    NotNanEnvironmentCountries=((RelevantYearsAndFeaturesEnvironment[RelevantYearsAndFeaturesEnvironment["Country Name"]==countries].drop(["Country Name", "Country Code", "Indicator", "Indicator Code"], axis=1).groupby("Features").count()))
    NotNanEnvironmentCountries['not_nan_values_count'] = NotNanEnvironmentCountries.apply(lambda x: x.sum(), axis=1)
    plt.figure(figsize=(20, 10))
    sns.heatmap(NotNanEnvironmentCountries.sort_values(by="not_nan_values_count", ascending=False).drop(["not_nan_values_count"], axis=1), cbar=True, linewidths=0.1)
    plt.title("Mapa de Calor: Proporción de Valores NaN por feature seleccionados y por año en " + countries)

#### 2 Cálculo de estadísticos y distribuciones

2.a. Obtener la media, mediana, moda, desviación estándar, varianza, valor máximo y mínimo, percentiles y cualquier otro estadístico que les resulte importante de cada uno de los features de cada Indicador. 

In [ ]:
# Calculamos estadisticos por feature para la totalidad de los años
CleanInitialEnvironmentByYear.drop("Year", axis=1).groupby(["Features"]).describe()

2.b. Realizar nuevamente los histogramas ó boxplots de las 15 features para cada Indicador.

In [ ]:
CleanInitialEnvironmentByYear = CleanInitialEnvironmentByYear.copy()

features = CleanInitialEnvironmentByYear["Features"].drop_duplicates().to_list()

for features in features:
    plt.figure(figsize=(10,6))
    sns.boxplot(data=CleanInitialEnvironmentByYear[CleanInitialEnvironmentByYear["Features"]==features], x="Features", y="value", width=0.1)
    plt.title("Boxplot por feature", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.ylabel('Valor', fontsize=16)

2.c. Trazar los gráficos de línea de los 15 features en el intervalo considerado.

In [ ]:
CleanInitialEnvironmentByYear = CleanInitialEnvironmentByYear.copy()

features = CleanInitialEnvironmentByYear["Features"].drop_duplicates().to_list()

for features in features:
    plt.figure(figsize=(10,6))
    sns.pointplot(data=CleanInitialEnvironmentByYear[CleanInitialEnvironmentByYear["Features"]==features], x="Year", y="value")
    plt.title("Serie Temporal de \n" + str(features), fontsize=16)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Año', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

2.d. Analizar en conjunto los resultados de los puntos a, b y c. 

De los puntos a, b y c se desprenden las siguientes apreciaciones:
- Los features son sumamente heterogeneos en cuanto a sus valores dado que representan diferentes conceptos; existen porcentajes varios y cantidades. 
- Cada feature es singular en cuanto al rango de sus datos, si bien algunos poseen más variabilidad en los datos por año que otros, son pocos los features que presentan outliers. Sólo 4 de los 15 features presentan valores atipicos.
- Las tendencias de los features son mayoritariamente distintas, sin poder generalizar un comportamiento para todos los features del indicador. Sin embargo, algunos features particulares, por grupos, presentan tendencias similares.
​

Tendencia creciente: 
- Tierras agrícolas (% del área de tierra)
- Tierras cultivables (% del área de tierra)
- Ahorro ajustado: gasto en educación (% del INB)
- Ahorro ajustado: daño por emisión de dióxido de carbono (US actuales)
- Ahorro ajustado: consumo de capital fijo (US actuales)
​

Tendencia creciente y luego decreciente:
- Rentas totales de los recursos naturales (% del PIB)
- Rentas mineras (% del PIB)
- Ahorro ajustado: agotamiento de minerales (US actuales)
​

Tendencia sumamente variable:
- Aqui se tienen dos indicadores con la misma tendencia cambiante; Producción de la pesca de captura (toneladas métricas) y Producción pesquera total (toneladas métricas)
- Ahorro ajustado: agotamiento neto de recursos forestales (US a precios actuales)
- Rentas forestales (% del PIB)
- Ahorro ajustado: agotamiento de fuentes de energía (US actuales)
​

Tendencia sin cambios
- Área de tierra (kilómetros cuadrados)
- Superficie (kilómetros cuadrados)

#### 3 Análisis de posible imputación de valores faltantes

3.a. En base al estudio y análisis anterior, ¿Considera que se dispone de suficiente información para completar/rellenar los valores faltantes? ¿Cuáles reemplazaría y qué criterio/s usaría? Justifique sus respuestas.

Dado que se dispone de la totalidad de datos previos a 2017, entonces se decide estimar tanto 2017 como 2018. No se descarta ningun feature.

3.b. Identificar los casos en los cuáles no se cuenta con información suficiente para compensar la falta de valores nulos y analizar dichos features valiendose del análisis y visualización realizada en el práctico anterior:
- ¿Cómo es la distribución de este feature en los demás países? 
- ¿Presentan la misma tendencia a lo largo del tiempo? ¿Presentan la misma cantidad de valores faltantes? 
- ¿Es posible establecer alguna relación entre los países vecinos a Argentina o con algún país en particular?
- ¿De qué manera la posible correlación con otros países se encuentra limitada por el contexto local? Por ejemplo: La crisis del 2001 en Argentina es un evento totalmente local que probablemente no haya afectado a la región).

No hay features que no se considerarán.

3.c y 3.d. Conclusiones de analisis y limpieza final

Como se adelantó, se completarán los valores faltantes para los features que corresponda de manera de trabajar con los años de 1990 a 2018.

Primero completamos 2017. Se toma la media ponderada de los ultimos tres años, con distinto peso cada uno de manera de dar mayor importancia al dato mas reciente.

In [ ]:
CleanEnvironmentStepOne = CleanInitialEnvironment.copy()

In [ ]:
# Seleccionamos features sin Nans en el 2017 para no estimar tal año para ellos
CleanEnvironmentStepOneNotNull = CleanEnvironmentStepOne[CleanEnvironmentStepOne["2017"].notnull()]
CleanEnvironmentStepOneNotNull

# Seleccionamos features con NaNs en el 2017 y renombramos columnas para poder seleccionarlas más rapidamente
CleanEnvironmentStepOneNull = CleanEnvironmentStepOne[CleanEnvironmentStepOne["2017"].isna()]
CleanEnvironmentStepOneNull = CleanEnvironmentStepOneNull.rename(columns={"2013":"T_Four", "2014":"T_Three", "2015":"T_Two", "2016":"T_One"})

# En caso que exista una tendencia creciente O decreciente, aplicamos la tasa de crecimiento promedio de los ultimos tres años para interpolar el dato faltante.
IncreasingTrend = ((CleanEnvironmentStepOneNull.T_One - CleanEnvironmentStepOneNull.T_Two) > 0) & ((CleanEnvironmentStepOneNull.T_Two - CleanEnvironmentStepOneNull.T_Three)>0) & ((CleanEnvironmentStepOneNull.T_Three - CleanEnvironmentStepOneNull.T_Four)>0)
DecreasingTrend = ((CleanEnvironmentStepOneNull.T_One - CleanEnvironmentStepOneNull.T_Two) < 0) & ((CleanEnvironmentStepOneNull.T_Two - CleanEnvironmentStepOneNull.T_Three)<0) & ((CleanEnvironmentStepOneNull.T_Three - CleanEnvironmentStepOneNull.T_Four)<0)

if ((IncreasingTrend)|(DecreasingTrend)) is True:
    GrowthRateEnvironmentThreePreviousPeriods = (CleanEnvironmentStepOneNull.T_Three - CleanEnvironmentStepOneNull.T_Four)/(CleanEnvironmentStepOneNull.T_Four)
    GrowthRateEnvironmentTwoPreviousPeriods = (CleanEnvironmentStepOneNull.T_Two - CleanEnvironmentStepOneNull.T_Three)/(CleanEnvironmentStepOneNull.T_Three)
    GrowthRateEnvironmentOnePreviousPeriods = (CleanEnvironmentStepOneNull.T_One - CleanEnvironmentStepOneNull.T_Two)/(CleanEnvironmentStepOneNull.T_Two)
    GrowthRateEnvironment = (GrowthRateEnvironmentThreePreviousPeriods+GrowthRateEnvironmentTwoPreviousPeriods+GrowthRateEnvironmentOnePreviousPeriods)/3
    CleanEnvironmentStepOneNull["2017"] = CleanEnvironmentStepOneNull.T_One*(1+GrowthRateEnvironment)

# Caso contrario, si no hay una tendencia clara antes de 2017, aplicamos un promedio ponderado de los ultimos tres años. Los datos más recientes tienen mas peso.
else:
    CleanEnvironmentStepOneNull["2017"] = CleanEnvironmentStepOneNull.T_Three*0.2 + CleanEnvironmentStepOneNull.T_Two*0.3 + CleanEnvironmentStepOneNull.T_One*0.5
    
# Unimos dataframe sin Nans, con datos originales para el 2017, con el dataframe sobre el que se realizó la estimación del valor de 2017.
CleanEnvironmentStepTwo = pd.concat([CleanEnvironmentStepOneNotNull, CleanEnvironmentStepOneNull.rename(columns={"T_Three":"2014", "T_Two":"2015", "T_One":"2016", "T_Four":"2013"})], ignore_index=False)
CleanEnvironmentStepTwo

Luego completamos 2018, con el mismo criterio que antes.

In [ ]:
# Seleccionamos features sin Nans en el 2018 para no estimar tal año para ellos
CleanEnvironmentStepTwoNotNull = CleanEnvironmentStepTwo[CleanEnvironmentStepTwo["2018"].notnull()]
CleanEnvironmentStepTwoNotNull

# Seleccionamos features con NaNs en el 2018 y renombramos columnas para poder seleccionarlas más rapidamente
CleanEnvironmentStepTwoNull = CleanEnvironmentStepTwo[CleanEnvironmentStepTwo["2018"].isna()]
CleanEnvironmentStepTwoNull = CleanEnvironmentStepTwoNull.rename(columns={"2014":"T_Four", "2015":"T_Three", "2016":"T_Two", "2017":"T_One"})

# En caso que exista una tendencia creciente O decreciente, aplicamos la tasa de crecimiento promedio de los ultimos tres años para interpolar el dato faltante.
IncreasingTrend = ((CleanEnvironmentStepTwoNull.T_One - CleanEnvironmentStepTwoNull.T_Two) > 0) & ((CleanEnvironmentStepTwoNull.T_Two - CleanEnvironmentStepTwoNull.T_Three)>0) & ((CleanEnvironmentStepTwoNull.T_Three - CleanEnvironmentStepTwoNull.T_Four)>0)
DecreasingTrend = ((CleanEnvironmentStepTwoNull.T_One - CleanEnvironmentStepTwoNull.T_Two) < 0) & ((CleanEnvironmentStepTwoNull.T_Two - CleanEnvironmentStepTwoNull.T_Three)<0) & ((CleanEnvironmentStepTwoNull.T_Three - CleanEnvironmentStepTwoNull.T_Four)<0)

if ((IncreasingTrend)|(DecreasingTrend)) is True:
    GrowthRateEnvironmentThreePreviousPeriods = (CleanEnvironmentStepTwoNull.T_Three - CleanEnvironmentStepTwoNull.T_Four)/(CleanEnvironmentStepTwoNull.T_Four)
    GrowthRateEnvironmentTwoPreviousPeriods = (CleanEnvironmentStepTwoNull.T_Two - CleanEnvironmentStepTwoNull.T_Three)/(CleanEnvironmentStepTwoNull.T_Three)
    GrowthRateEnvironmentOnePreviousPeriods = (CleanEnvironmentStepTwoNull.T_One - CleanEnvironmentStepTwoNull.T_Two)/(CleanEnvironmentStepTwoNull.T_Two)
    GrowthRateEnvironment = (GrowthRateEnvironmentThreePreviousPeriods+GrowthRateEnvironmentTwoPreviousPeriods+GrowthRateEnvironmentOnePreviousPeriods)/3
    CleanEnvironmentStepOneNull["2018"] = CleanEnvironmentStepTwoNull.T_One*(1+GrowthRateEnvironment)

# Caso contrario, si no hay una tendencia clara antes de 2018, aplicamos un promedio ponderado de los ultimos tres años. Los datos más recientes tienen mas peso.
else:
    CleanEnvironmentStepTwoNull["2018"] = CleanEnvironmentStepTwoNull.T_Three*0.2 + CleanEnvironmentStepTwoNull.T_Two*0.3 + CleanEnvironmentStepTwoNull.T_One*0.5
    
# Unimos dataframe sin Nans, con datos originales para el 2018, con el dataframe sobre el que se realizó la estimación del valor de 2018.
CleanEnvironmentStepThree = pd.concat([CleanEnvironmentStepTwoNotNull, CleanEnvironmentStepTwoNull.rename(columns={"T_Three":"2015", "T_Two":"2016", "T_One":"2017", "T_Four":"2014"})], ignore_index=False)
CleanEnvironmentStepThree

3.e. Dataset Final

In [ ]:
CleanEnvironmentStepThree.to_csv("Clean Environment")

## Presentación de resultados 

Realizar un informe en formato notebook, o el que se considere más apropiado, en la cual se presenten los resultados del análisis realizado. El mismo debe estar pensado para un público técnico pero que desconoce los aspectos propios del problema a resolver (por ejemplo, sus compañeros de clase). 

Se evaluará principalmente la claridad del mensaje presentado, el uso de las herramientas y los conceptos desarrollados durante las clases de la materia "Análisis y Curación". Asimismo, el mismo debe incluir de manera clara los hallazgos y problemas encontrados y las posibles accionar a tomar.

Se aconseja que el informe tenga una estructura ordenada, clara y fácil de leer. Por ejemplo, podría contar con una introducción a la problemática a resolver, una descripción de los datos a emplear, el desarrollo propiamente dicho de todo el trabajo y las conclusiones que se obtuvieron.


## Fechas de entrega

- 18/07/2020: Reunión para mostrar avances, resolver dudas y afinar cuestiones del análisis.
- 23/07/2020: Entrega del práctico. 